In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from pandas.api.types import CategoricalDtype
from collections import Counter
from itertools import combinations
import scipy.stats as stats
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import figure, show
from ipywidgets import interact, widgets
from bokeh.palettes import Set2, Set3
from scipy.stats import shapiro, kstest
from scipy.stats import boxcox
from statsmodels.graphics.mosaicplot import mosaic
from IPython.display import display, HTML
from scipy.stats import chi2_contingency
from statsmodels.formula.api import ols, logit
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve
)
from sklearn.preprocessing import LabelEncoder

#display all the contents
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None) 

#Ignore SettingWithCopyWarnings
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

Loading BokehJS ...

In [2]:
#import raw datasets
r_LC = pd.read_csv("Raw Datasets/1_LC.csv")
r_LR = pd.read_csv("Raw Datasets/2_LR.csv")
r_LG = pd.read_csv("Raw Datasets/3_LG.csv")
r_MC = pd.read_csv("Raw Datasets/4_MC.csv")
r_MR = pd.read_csv("Raw Datasets/5_MR.csv")
r_MG = pd.read_csv("Raw Datasets/6_MG.csv")
r_FC = pd.read_csv("Raw Datasets/7_FC.csv")
r_FR = pd.read_csv("Raw Datasets/8_FR.csv")
r_FG = pd.read_csv("Raw Datasets/9_FG.csv")

In [3]:
#Raw datasets lists
A_r = [r_LC, r_LR, r_LG, r_MC, r_MR, r_MG, r_FC, r_FR, r_FG]

L_r = [r_LC, r_LR, r_LG]
M_r = [r_MC, r_MR, r_MG]
F_r = [r_FC, r_FR, r_FG]

C_r = [r_LC, r_MC, r_FC]
R_r = [r_LR, r_MR, r_FR]
G_r = [r_LG, r_MG, r_FG]

In [4]:
unwanted_columns = ['StartDate', 'EndDate', 'Status', 'IPAddress',
       'Finished', 'RecordedDate', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q_RecaptchaScore', 'FERT-3_NPS_GROUP']

#delete unwanted columns
#Note that this function will change the value of the original dataframe directly
def del_unwanted_columns(datasets, unwanted_columns):
    if isinstance(datasets, pd.DataFrame):
        datasets = [datasets]

    for i, dataset in enumerate(datasets):
        dataset.drop(columns=unwanted_columns, inplace=True, errors='ignore')

del_unwanted_columns(A_r, unwanted_columns)

In [5]:
def remove_all_spaces(datasets):
    if isinstance(datasets, pd.DataFrame):
        datasets = [datasets]

    for dataset in datasets:
        dataset.columns = dataset.columns.str.replace(r"\s+", "", regex=True)
        
        for col in dataset.select_dtypes(include=["object"]).columns:
            dataset[col] = dataset[col].str.strip()

remove_all_spaces(A_r)

In [6]:
coding_DEMO = {"DEMO-1": "GEN", "DEMO-2": "AGE", "DEMO-3": "FACULTY", "DEMO-4": "RACE", "DEMO-4_8_TEXT": "RACE_Other", 
               "DEMO-5": "BIRTH_PLACE", "DEMO-5_5_TEXT": "BIRTH_PLACE_Other", "DEMO-6": "HAVE_BELIEF",
               "DEMO-7": "RELIGION", "DEMO-7_9_TEXT": "RELIGION_Other", "DEMO-8": "MARR_STATUS", "DEMO-8_8_TEXT": "MARR_STATUS_Other",
               "DEMO-9": "BIOKIDS", 'DEMO-9_4_TEXT': 'BIOKIDS_Other',
               "DEMO-10": "HAVE_SIBLINGS", "DEMO-11": "BIRTH_ORDER", "DEMO-11_6_TEXT": "BIRTH_ORDER_Other",
               "DEMO-12": "WHO_SIBLINGS", "DEMO-12_6_TEXT": "WHO_SIBLINGS_Other", "DEMO-13": "BIO_SIBLINGS", 
               "DEMO-13_5_TEXT": "BIO_SIBLINGS_Other", "DEMO-14": "CLOSE_FAMILY", "DEMO-14_6_TEXT": "CLOSE_FAMILY_Other",
               "DEMO-15": "CLOSE_FAMILY_NS", "DEMO-15_4_TEXT": "CLOSE_FAMILY_NS_Other", "DEMO-16_1": "FATHER_EDU", 
               "DEMO-16_2": "MOTHER_EDU", "DEMO-17": "INCOME_SOURCE", "DEMO-17_7_TEXT": "INCOME_SOURCE_Other",
               "DEMO-18_1": "INCOME",
               #Some of the columns of M part has been changed previously
               "RACE_8_TEXT": "RACE_Other", "BIRTH_PLACE_5_TEXT": "BIRTH_PLACE_Other",
               "RELIGION_9_TEXT": "RELIGION_Other", "MARR_STATUS_8_TEXT": "MARR_STATUS_Other",
               "BIRTH_ORDER_6_TEXT": "BIRTH_ORDER_Other", "WHO_SIBLINGS_6_TEXT": "WHO_SIBLINGS_Other",
               "BIO_SIBLINGS_5_TEXT": "BIO_SIBLINGS_Other", "CLOSE_FAMILY_6_TEXT": "CLOSE_FAMILY_Other" ,
               "CLOSE_FAMILY_NS_4_TEXT": "CLOSE_FAMILY_NS_Other", "PAR_EDU_1": "FATHER_EDU", "PAR_EDU_2": "MOTHER_EDU",
               "INCOME_SOURCE_7_TEXT": "INCOME_SOURCE_Other", "INCOME_1": "INCOME", 'BIOKIDS_4_TEXT': 'BIOKIDS_Other'
              }

coding_LOVE = {"LOVE-1": "R_STATUS","LOVE-1_6_TEXT": "R_STATUS_Other","LOVE-2": "TIMES", "LOVE-3": "IDEA_AGE", "LOVE-4": "FIRST_AGE", 
                "LOVE-5": "PURPOSE","LOVE-5_8_TEXT": "PURPOSE_Other" ,"LOVE-6": "REASON","LOVE-6_7_TEXT": "REASON_Other",
                "LOVE-7": "CRITERIA", "LOVE-7_9_TEXT": "CRITERIA_Other",
                "LOVE-8": "VIEW", "LOVE-8_6_TEXT": "VIEW_Other",
                "LOVE-9": "EXPECTATION_SINGLE", "LOVE-9_6_TEXT": "EXPECTATION_SINGLE_Other",
                "LOVE-10": "EXPECTATION", "LOVE-10_6_TEXT": "EXPECTATION_Other",
                "LOVE-11": "AFFECT", "LOVE-11_6_TEXT": "AFFECT_Other",
                "LOVE-12-16_1": "EVALUATE_MAN", 
                "LOVE-12-16_2": "EVALUATE_BESELF", 
                "LOVE-12-16_3": "EVALUATE_NEGOTIATE", 
                "LOVE-12-16_4": "EVALUATE_EMO", 
                "LOVE-12-16_5": "EVALUATE_HOPE", 
               }

coding_MARR = {
    "MAR_STATUS": "R_STATUS",
    "MAR_STATUS_6_TEXT": "R_STSTUS_Other",
    "WILLING_MAR_7_TEXT": "WILLING_MAR_Other",
    "MAR_PERS_1": "LOVE_IN_MAR",
    "MAR_PERS_2": 'FIN_IN_MAR',
    "MAR_PERS_3": "TRAD_IN_MAR",
    "MAR_PERS_4": "COR_IN_MRA",
    "MAR_PERS_5": "FULFIL_IN_MRA",
    "MAR_FACTORS_1": "ECO",
    "MAR_FACTORS_2": "COM",
    "MAR_FACTORS_3": "PER",
    "MAR_FACTORS_4": "EMO",
    "MAR_FACTORS_5": "INT",
    "MAR_FACTORS_6": "LIF",
    "BEFORE_SEX_6_TEXT": "BEFORE_SEX_Other",
    "PARTNER_BEFORE_SEX_6_TEXT": "PARTNER_BEFORE_SEX_Other",
    "R_BEFORE_SEX_6_TEXT": "R_BEFORE_SEX_Other",
    "ARRANGED_MAR_6_TEXT": "ARRANGED_MAR_Other",
    "EXCPET_MAR_6_TEXT": "EXCPET_MAR_Other",
    "MAR_AGE_5_TEXT": "'MAR_AGE_5_Other",
    'PARTNER_BEFORE_SEX_12_TEXT': "PARTNER_BEFORE_SEX_Other",
    'R_BEFORE_SEX_15_TEXT': 'R_BEFORE_SEX_Other',
    'ARRANGED_MAR_13_TEXT': 'ARRANGED_MAR_Other',
    'EXCPET_MAR_13_TEXT': 'EXCPET_MAR_Other'
}

coding_FERT = {
    'FERT-0': 'R_STATUS', 'FERT-0_6_TEXT': 'R_STATUS_Other', 'FERT-1': 'PLAN_FERT',
    'FERT-2': 'PARTNER_PLAN_FERT', 'FERT-3': 'SCALE_PLAN_FERT', 'FERT-4': 'CHILD_NUM', 'FERT-4_7_TEXT': 'CHILD_NUM_Other',
    'FERT-5': 'DISCUSS_CHILD', 'FERT-6_1': 'BEST_FERT_AGE', 'FERT-7': 'CHILD_GEN', 'FERT-7_5_TEXT': 'CHILD_GEN_Other', 'FERT-8': 'FACTOR_FERT_PLAN',
    'FERT-8_10_TEXT': 'FACTOR_FERT_PLAN_Other', 'FERT-9': 'FERT_PURPOSE', 'FERT-9_9_TEXT': 'FERT_PURPOSE_Other', 'FERT-10': 'FERT_VALUE', 'FERT-11': 'FERT_CAREER',
    'FERT-12': 'WHO_PARENT', 'FERT-12_6_TEXT': 'WHO_PARENT_Other', 'FERT-13': 'CHILD_NUM_INFLUENCE', 'FERT-14': 'PARENT_RESPONS', 'FERT-14_6_TEXT': 'PARENT_RESPONS_Other',
    'FERT-15': 'FACTOR_FERT', 'FERT-15_6_TEXT': 'FACTOR_FERT_Other'
}

coding_CULT = {
       'CULT-1': 'CULT_MARR_ATT', 'CULT-2': 'CULT_FERT_ATT', 'CULT-3': 'FAMILY_IMPACT',
       'CULT-4': 'WEDDING', 'CULT-5-13_1': 'FAMILY_BOOK', 'CULT-5-13_2': 'FAMILY_RESOURCE', 'CULT-5-13_3': 'PARICIPATION', 
       'CULT-5-13_4': 'STUDY_HABIT', 'CULT-5-13_5': 'MEDIA', 'CULT-5-13_6': 'SOCIAL', 'CULT-5-13_7': 'TOPIC', 'CULT-5-13_8': 'DIVERSITY',
    'CULT-5-18_1': 'FAMILY_BOOK', 'CULT-5-18_2': 'FAMILY_RESOURCE', 'CULT-5-18_3': 'PARICIPATION', 'CULT-5-18_4': 'STUDY_HABIT',
       'CULT-5-18_5': 'MEDIA', 'CULT-5-18_6': 'SOCIAL', 'CULT-5-18_7': 'TOPIC', 'CULT-5-18_8': 'DIVERSITY'
}

coding_RCFE = {
    'RCFE-1': 'FAM_EDU', 'RCFE-2': 'PAR_MARR', 'RCFE-3': 'CHILDHOOD',
       'RCFE-4-11_1': 'TRAD_IMPORTANCE', 'RCFE-4-11_2': 'IDENTITY', 'RCFE-4-11_3': 'CULT_CONFIDENCE', 'RCFE-4-11_4': 'FAMILY_INFLUENCE',
       'RCFE-4-11_5': 'FAM_ADVICE', 'RCFE-4-11_6': 'ON_PATENER_ADVICE', 'RCFE-4-11_7': 'TRAD_MAR', 'RCFE-4-11_8': 'PRIORITY',
       'RCFE-4-21_1': 'TRAD_IMPORTANCE', 'RCFE-4-21_2': 'IDENTITY', 'RCFE-4-21_3': 'CULT_CONFIDENCE', 'RCFE-4-21_4': 'FAMILY_INFLUENCE',
       'RCFE-4-21_5': 'FAM_ADVICE', 'RCFE-4-21_6': 'ON_PATENER_ADVICE', 'RCFE-4-21_7': 'TRAD_MAR', 'RCFE-4-21_8': 'PRIORITY'
}

coding_GDRO = {
    'GDRO-1': 'PAR_EVEN', 'GDRO-2-11_1': 'ECO_MALE', 'GDRO-2-11_2': 'DECISION_MALE',
       'GDRO-2-11_3': 'PAR_BOTH', 'GDRO-2-11_4': 'LABOUR', 'GDRO-2-11_5': 'MALE_LABOUR', 'GDRO-2-11_6': 'WORKING_GDRO',
       'GDRO-2-11_7': 'ABS_EQUAL', 'GDRO-2-11_8': 'FE_ALL', 'GDRO-2-11_9': 'PREJ',
    'GDRO-2-_1': 'ECO_MALE',
       'GDRO-2-_2': 'DECISION_MALE', 'GDRO-2-_3': 'PAR_BOTH', 'GDRO-2-_4': 'LABOUR', 'GDRO-2-_5': 'MALE_LABOUR', 'GDRO-2-_6': 'WORKING_GDRO',
       'GDRO-2-_7': 'ABS_EQUAL', 'GDRO-2-_8': 'FE_ALL', 'GDRO-2-_9': 'PREJ',
    'GDRO-2-10_1': 'ECO_MALE', 'GDRO-2-10_2': 'DECISION_MALE', 'GDRO-2-10_3': 'PAR_BOTH', 'GDRO-2-10_4': 'LABOUR',
       'GDRO-2-10_5': 'MALE_LABOUR', 'GDRO-2-10_6': 'WORKING_GDRO', 'GDRO-2-10_7': 'ABS_EQUAL', 'GDRO-2-10_8': 'FE_ALL',
       'GDRO-2-10_9': 'PREJ'
}

In [7]:
def coding_datasets(datasets, rule):
    if isinstance(datasets, pd.DataFrame):
        datasets = [datasets]
        
    for dataset in datasets:
        dataset.rename(columns=rule, inplace=True)

coding_datasets(A_r, coding_DEMO)

coding_datasets(L_r, coding_LOVE)
coding_datasets(M_r, coding_MARR)
coding_datasets(F_r, coding_FERT)

coding_datasets(C_r, coding_CULT)
coding_datasets(R_r, coding_RCFE)
coding_datasets(G_r, coding_GDRO)

In [8]:
GENERAL = ['Duration(inseconds)', 'Progress', 'ResponseId']

DEMO = GENERAL + ['GEN', 'AGE', 'FACULTY', 'RACE',
       'RACE_Other', 'BIRTH_PLACE', 'BIRTH_PLACE_Other', 'HAVE_BELIEF',
       'RELIGION', 'RELIGION_Other', 'MARR_STATUS', 'MARR_STATUS_Other',
       'BIOKIDS', 'BIOKIDS_Other', 'HAVE_SIBLINGS', 'BIRTH_ORDER', 'BIRTH_ORDER_Other',
       'WHO_SIBLINGS', 'WHO_SIBLINGS_Other', 'BIO_SIBLINGS',
       'BIO_SIBLINGS_Other', 'CLOSE_FAMILY', 'CLOSE_FAMILY_Other',
       'CLOSE_FAMILY_NS', 'CLOSE_FAMILY_NS_Other', 'FATHER_EDU', 'MOTHER_EDU',
       'INCOME_SOURCE', 'INCOME_SOURCE_Other', 'INCOME']

LOVE = GENERAL + ['R_STATUS', 'R_STATUS_Other', 'TIMES', 'IDEA_AGE', 'FIRST_AGE',
       'PURPOSE', 'PURPOSE_Other', 'REASON', 'REASON_Other',
       'CRITERIA', 'CRITERIA_Other', 'VIEW', 'VIEW_Other',
       'EXPECTATION_SINGLE', 'EXPECTATION_SINGLE_Other', 'EXPECTATION', 'EXPECTATION_Other', 'AFFECT',
       'AFFECT_Other', 'EVALUATE_MAN', 'EVALUATE_BESELF',
       'EVALUATE_NEGOTIATE', 'EVALUATE_EMO', 'EVALUATE_HOPE',
     ]

FERT = GENERAL + ['R_STATUS', 'R_STATUS_Other',
       'PLAN_FERT', 'PARTNER_PLAN_FERT', 'SCALE_PLAN_FERT', 'CHILD_NUM',
       'CHILD_NUM_Other', 'DISCUSS_CHILD', 'BEST_FERT_AGE', 'CHILD_GEN',
       'CHILD_GEN_Other', 'FACTOR_FERT_PLAN', 'FACTOR_FERT_PLAN_Other',
       'FERT_PURPOSE', 'FERT_PURPOSE_Other', 'FERT_VALUE', 'FERT_CAREER',
       'WHO_PARENT', 'WHO_PARENT_Other', 'CHILD_NUM_INFLUENCE',
       'PARENT_RESPONS', 'PARENT_RESPONS_Other', 'FACTOR_FERT',
       'FACTOR_FERT_Other']

MARR = GENERAL + ['R_STATUS', 'R_STSTUS_Other',
       'WILLING_MAR', 'WILLING_MAR_Other', 'MAR_AGE', 'MAR_CHOICE',
       'MAR_IMPORTANCE', 'LOVE_IN_MAR', 'FIN_IN_MAR', 'TRAD_IN_MAR',
       'COR_IN_MRA', 'FULFIL_IN_MRA', 'ECO', 'COM', 'PER', 'EMO', 'INT', 'LIF',
       'BEFORE_SEX', 'PARTNER_BEFORE_SEX',
       'PARTNER_BEFORE_SEX_Other', 'R_BEFORE_SEX', 'R_BEFORE_SEX_Other',
       'ARRANGED_MAR', 'ARRANGED_MAR_Other', 'EXCPET_MAR', 'EXCPET_MAR_Other']

CULT = GENERAL + ['CULT_MARR_ATT', 'CULT_FERT_ATT', 'FAMILY_IMPACT', 'WEDDING',
       'FAMILY_BOOK', 'FAMILY_RESOURCE', 'PARICIPATION', 'STUDY_HABIT',
       'MEDIA', 'SOCIAL', 'TOPIC', 'DIVERSITY']

RCFE = GENERAL + ['FAM_EDU', 'PAR_MARR', 'CHILDHOOD',
       'TRAD_IMPORTANCE', 'IDENTITY', 'CULT_CONFIDENCE', 'FAMILY_INFLUENCE',
       'FAM_ADVICE', 'ON_PATENER_ADVICE', 'TRAD_MAR', 'PRIORITY']

GDRO = GENERAL + ['PAR_EVEN', 'ECO_MALE',
       'DECISION_MALE', 'PAR_BOTH', 'LABOUR', 'MALE_LABOUR', 'WORKING_GDRO',
       'ABS_EQUAL', 'FE_ALL', 'PREJ']

CTRL_1 = DEMO
CTRL_2 = list(set(CULT + RCFE + GDRO))

In [9]:
def get_combined(datasets, rule):
    if not isinstance(datasets, list):
        datasets = [datasets]
    filtered_datasets = []
    
    for dataset in datasets:
        filtered_datasets.append(dataset[rule])

    combined_dataset = pd.concat(filtered_datasets, ignore_index=True)

    return combined_dataset

DEMO_dataset = get_combined(A_r, DEMO)

LOVE_dataset = get_combined(L_r, LOVE)
MARR_dataset = get_combined(M_r, MARR)
FERT_dataset = get_combined(F_r, FERT)

CULT_dataset = get_combined(C_r, CULT)
RCFE_dataset = get_combined(R_r, RCFE)
GDRO_dataset = get_combined(G_r, GDRO)

In [10]:
def remove_non_numeric_rows(dataset, column_name):
    if column_name not in dataset.columns:
        raise ValueError(f"Column '{column_name}' not found in the dataset.")

    dataset[column_name] = pd.to_numeric(dataset[column_name], errors='coerce')
    dataset.dropna(subset=[column_name], inplace=True)
    dataset.reset_index(drop=True, inplace=True)

#Duration should never be NaN
remove_non_numeric_rows(DEMO_dataset, "Duration(inseconds)")
remove_non_numeric_rows(LOVE_dataset, "Duration(inseconds)")
remove_non_numeric_rows(MARR_dataset, "Duration(inseconds)")
remove_non_numeric_rows(FERT_dataset, "Duration(inseconds)")
remove_non_numeric_rows(CULT_dataset, "Duration(inseconds)")
remove_non_numeric_rows(RCFE_dataset, "Duration(inseconds)")
remove_non_numeric_rows(GDRO_dataset, "Duration(inseconds)")

In [11]:
DEMO_dataset["Progress"] = pd.to_numeric(DEMO_dataset["Progress"], errors="coerce")
DEMO_dataset = DEMO_dataset[(DEMO_dataset["Progress"] >= 30) & (DEMO_dataset["Duration(inseconds)"] >= 60)]
DEMO_dataset.reset_index(drop=True, inplace=True)

LOVE_dataset["Progress"] = pd.to_numeric(LOVE_dataset["Progress"], errors="coerce")
LOVE_dataset = LOVE_dataset[(LOVE_dataset["Progress"] >= 30) & (LOVE_dataset["Duration(inseconds)"] >= 60)]
LOVE_dataset.reset_index(drop=True, inplace=True)

MARR_dataset["Progress"] = pd.to_numeric(MARR_dataset["Progress"], errors="coerce")
MARR_dataset = MARR_dataset[(MARR_dataset["Progress"] >= 30) & (MARR_dataset["Duration(inseconds)"] >= 60)]
MARR_dataset.reset_index(drop=True, inplace=True)

FERT_dataset["Progress"] = pd.to_numeric(FERT_dataset["Progress"], errors="coerce")
FERT_dataset = FERT_dataset[(FERT_dataset["Progress"] >= 30) & (FERT_dataset["Duration(inseconds)"] >= 60)]
FERT_dataset.reset_index(drop=True, inplace=True)

CULT_dataset["Progress"] = pd.to_numeric(CULT_dataset["Progress"], errors="coerce")
CULT_dataset = CULT_dataset[(CULT_dataset["Progress"] >= 30) & (CULT_dataset["Duration(inseconds)"] >= 60)]
CULT_dataset.reset_index(drop=True, inplace=True)

RCFE_dataset["Progress"] = pd.to_numeric(RCFE_dataset["Progress"], errors="coerce")
RCFE_dataset = RCFE_dataset[(RCFE_dataset["Progress"] >= 30) & (RCFE_dataset["Duration(inseconds)"] >= 60)]
RCFE_dataset.reset_index(drop=True, inplace=True)

GDRO_dataset["Progress"] = pd.to_numeric(GDRO_dataset["Progress"], errors="coerce")
GDRO_dataset = GDRO_dataset[(GDRO_dataset["Progress"] >= 30) & (GDRO_dataset["Duration(inseconds)"] >= 60)]
GDRO_dataset.reset_index(drop=True, inplace=True)

In [12]:
def remove_empty_columns(dataset):
    dataset.dropna(axis=1, how="all", inplace=True)
    return None

remove_empty_columns(DEMO_dataset)
remove_empty_columns(LOVE_dataset)
remove_empty_columns(MARR_dataset)
remove_empty_columns(FERT_dataset)
remove_empty_columns(CULT_dataset)
remove_empty_columns(RCFE_dataset)
remove_empty_columns(GDRO_dataset)

In [13]:
def handle_other_columns(dataset):
    for col in dataset.columns:
        if col.endswith("_Other"):
            main_col = col.replace("_Other", "")

            if main_col in dataset.columns:
                mask = dataset[col].notna()
                dataset.loc[mask, main_col] = dataset.loc[mask, col]

            dataset.drop(columns=col, inplace=True)

handle_other_columns(DEMO_dataset)
handle_other_columns(LOVE_dataset)
handle_other_columns(MARR_dataset)
handle_other_columns(FERT_dataset)
handle_other_columns(CULT_dataset)
handle_other_columns(RCFE_dataset)
handle_other_columns(GDRO_dataset)

In [14]:
DEMO_dataset["AGE"] = pd.to_numeric(DEMO_dataset["AGE"], errors="coerce")
DEMO_dataset["INCOME"] = pd.to_numeric(DEMO_dataset["INCOME"], errors="coerce")

#Mapping of answers
GEN_map = {'男': 'Male', '女': 'Female', '非二元性別/第三性別': 'Non-binary/Third Gender', '不願透露': 'Prefer not to say'}
DEMO_dataset.loc[:, "GEN"] = DEMO_dataset["GEN"].replace(GEN_map)

FACULTY_map = {'教育學院 FED': 'FED' ,
               '其他': 'Other', '科技學院 FST': 'FST', 
               '健康科學學院 FHS': 'FHS', 
               '微電子研究院 IME': 'IME', '工商管理學院 FBA': 'FBA',
               '社會科學學院 FSS': 'FSS', '法學院 FLL': 'FLL',
               '人文學院 FAH': 'FAH',
               'Faculty of Arts and Humanities': 'FAH', 'Faculty of Health Sciences': 'FHS',
               'Faculty of Science and Technology': 'FST', 'Faculty of Education': 'FED',
               'Faculty of Business Administration': 'FBA', 'Faculty of the Law': 'FLL',
               'Faculty of Social Sciences': 'FSS', 'Institute of Applied Physics and Materials Engineering': 'IAPME',
               'Institute of Collaborative Innovation': 'ICI',
               'Institute of Chinese Medical Sciences': 'ICMS',
               'Asia-Pacific Academy of Economics and Management': 'APAEM',
               'Institute of Microelectronics': 'IME'}
DEMO_dataset.loc[:, "FACULTY"] = DEMO_dataset["FACULTY"].replace(FACULTY_map)

RACE_map = {'華人': 'Chinese'}
DEMO_dataset.loc[:, "RACE"] = DEMO_dataset["RACE"].replace(RACE_map)

BIRTH_PLACE_map = {'中國大陸': 'Mainland China', '香港/澳門': 'Hong Kong/Macau', 
                   '不願透露': 'Prefer not to say', '阿根廷': 'Argentina'}
DEMO_dataset.loc[:, "BIRTH_PLACE"] = DEMO_dataset["BIRTH_PLACE"].replace(BIRTH_PLACE_map)

HAVE_BELIEF_map = {'否': 'No', '不願透露': 'Prefer not to say', '是': 'Yes'}
DEMO_dataset.loc[:, "HAVE_BELIEF"] = DEMO_dataset["HAVE_BELIEF"].replace(HAVE_BELIEF_map)

RELIGION_map = {'佛教': 'Buddhism', '天主教': 'Catholicism'}
DEMO_dataset.loc[:, "RELIGION"] = DEMO_dataset["RELIGION"].replace(RELIGION_map)

MARR_STATUS_map = {'父母結婚': 'Parents are married', '父母分居': 'Parents are separated', '不願透露': 'Prefer not to say'}
DEMO_dataset.loc[:, "MARR_STATUS"] = DEMO_dataset["MARR_STATUS"].replace(MARR_STATUS_map)

BIOKIDS_map = {'Don’t have a kid': np.nan, '以生物學上的子女身份出生': 'Born as a biological child'} #Obviously a wrong response
DEMO_dataset.loc[:, "BIOKIDS"] = DEMO_dataset["BIOKIDS"].replace(BIOKIDS_map)

HAVE_SIBLINGS_map = {'有兄弟姐妹': 'Have siblings', '無兄弟姐妹': 'No siblings' , '不願透露': 'Prefer not to say'}
DEMO_dataset.loc[:, "HAVE_SIBLINGS"] = DEMO_dataset["HAVE_SIBLINGS"].replace(HAVE_SIBLINGS_map)

BIRTH_ORDER_map = {'長子/長女': 'Oldest son/daughter',
       '次子/次女': 'Middle son/daughter', '幼子/幼女': 'Youngest son/daughter', 'twins': 'Twins', '两个同父异母的姐姐': 'Youngest son/daughter',
                  'Only child': np.nan}
DEMO_dataset.loc[:, "BIRTH_ORDER"] = DEMO_dataset["BIRTH_ORDER"].replace(BIRTH_ORDER_map)

WHO_SIBLINGS_map = {'哥哥': 'Older brother',
    '弟弟': 'Younger brother', '弟弟,妹妹': 'Younger brother,Younger sister',
    '妹妹': 'Younger sister',
    '两个姐姐都在美国留学': 'Older sister,Older sister',
    'two younger half sisters': 'Younger sister',
    '同父異母的妹妹': 'Younger sister',
    '不願透露': 'Prefer not to say',
    'twins': np.nan, 'NO': 'Prefer not to say'}
DEMO_dataset.loc[:, "WHO_SIBLINGS"] = DEMO_dataset["WHO_SIBLINGS"].replace(WHO_SIBLINGS_map)

BIO_SIBLINGS_map = {'NO': 'Prefer not to say', '全部以生物學上的子女身份出生': 'All were born as biological children'}
DEMO_dataset.loc[:, "BIO_SIBLINGS"] = DEMO_dataset["BIO_SIBLINGS"].replace(BIO_SIBLINGS_map)

CLOSE_FAMILY_map = {'parents': 'Father,Mother', '朋友': 'Prefer not to say', 
                    'Prefer not to say,I interact more closely with someone else (please specify below)': np.nan,
                    '都差不多': 'Prefer not to say', '奶奶': 'Grand parents', '都一样': 'Prefer not to say',
                    'I interact more closely with someone else (please specify below)': np.nan, 'My friend': np.nan,
                    '不願透露': 'Prefer not to say', '父親,母親': 'Father,Mother', 
                    '姐姐/妹妹,哥哥/弟弟': 'Older sister/younger sister,Older brother/younger brother', 
                    '母親': 'Mother', '哥哥/弟弟': 'Older brother/younger brother', '父親': 'Father',
       '父親,母親,姐姐/妹妹': 'Father,Mother,Older sister/younger sister', '表弟，表哥':'Cousins'}
DEMO_dataset.loc[:, "CLOSE_FAMILY"] = DEMO_dataset["CLOSE_FAMILY"].replace(CLOSE_FAMILY_map)

CLOSE_FAMILY_NS_map = {'parents': 'Father,Mother', '朋友': 'Prefer not to say', 
                    '伴侶': np.nan, '外婆': 'Grand parents',
                    '都差不多': 'Prefer not to say', 'grandma': 'Grand parents', '都很密切': 'Prefer not to say',
                    'I interact more closely with someone else (please specify below)': np.nan, '朋友': np.nan,
                    '不願透露': 'Prefer not to say', '父親,母親': 'Father,Mother', 
                    '姐姐/妹妹,哥哥/弟弟': 'Older sister/younger sister,Older brother/younger brother', 
                    '母親': 'Mother', '哥哥/弟弟': 'Older brother/younger brother', '父親': 'Father',
                    'I interact more closely with someone else (please specify below)': np.nan, 
                    '我與其他人的互動更為密切（請在下方欄註明）': np.nan}
DEMO_dataset.loc[:, "CLOSE_FAMILY_NS"] = DEMO_dataset["CLOSE_FAMILY_NS"].replace(CLOSE_FAMILY_NS_map)

FATHER_EDU_map = {'中學': 'Secondary school', '博士學位': 'Doctoral degree', '小學': 'Primary school',
       '學士學位': "Bachelor's degree", '碩士學位': "Master's degree"}
DEMO_dataset.loc[:, "FATHER_EDU"] = DEMO_dataset["FATHER_EDU"].replace(FATHER_EDU_map)


MOTHER_EDU_map = {'中學': 'Secondary school', '博士學位': 'Doctoral degree', '小學': 'Primary school',
       '學士學位': "Bachelor's degree", '碩士學位': "Master's degree"}
DEMO_dataset.loc[:, "MOTHER_EDU"] = DEMO_dataset["MOTHER_EDU"].replace(MOTHER_EDU_map)

INCOME_SOURCE_map = {'家庭支持（父母或其他家庭成員提供）,工作收入（兼職或全職）': 'Family support (provided by parents or other family members),Income from work (part-time or full-time)', 
                  '家庭支持（父母或其他家庭成員提供）,獎學金或助學金': 'Family support (provided by parents or other family members),Scholarships or financial aid',
                  '家庭支持（父母或其他家庭成員提供）': 'Family support (provided by parents or other family members)', 
                  '不願透露': 'Prefer not to say', '儲蓄或個人存款': 'Savings or personal savings',
                  '家庭支持（父母或其他家庭成員提供）,獎學金或助學金,工作收入（兼職或全職）,儲蓄或個人存款':'Family support (provided by parents or other family members),Scholarships or financial aid,Income from work (part-time or full-time),Savings or personal savings',
                  '工作收入（兼職或全職）': 'Income from work (part-time or full-time)',
                  '家庭支持（父母或其他家庭成員提供）,獎學金或助學金,工作收入（兼職或全職）': 'Family support (provided by parents or other family members),Scholarships or financial aid,Income from work (part-time or full-time)',
                  '家庭支持（父母或其他家庭成員提供）,工作收入（兼職或全職）,儲蓄或個人存款': 'Family support (provided by parents or other family members),Income from work (part-time or full-time),Savings or personal savings', 
                  '家庭支持（父母或其他家庭成員提供）,助學貸款': 'Family support (provided by parents or other family members),Student loans',
                  '投資': 'Investment'
                   }
DEMO_dataset.loc[:, "INCOME_SOURCE"] = DEMO_dataset["INCOME_SOURCE"].replace(INCOME_SOURCE_map)

#Standarize for shrter labels for INCOME_SOURCE
INCOME_SOURCE_map = {
                  'Family support (provided by parents or other family members)': 'Family support',
                  'Income from work (part-time or full-time)': 'Salary',
                  'Family support (provided by parents or other family members),Income from work (part-time or full-time),Savings or personal savings': 'Family support,Salary,Savings',
                  'Family support (provided by parents or other family members),Income from work (part-time or full-time)': 'Family support,Salary',
                  'Family support (provided by parents or other family members),Scholarships or financial aid,Income from work (part-time or full-time)': 'Family support,Scholarship,Salary',
                  'Family support (provided by parents or other family members),Prefer not to say': 'Family support',
                  'Family support (provided by parents or other family members),Savings or personal savings': 'Family support,Savings',
                  'Income from work (part-time or full-time),Student loans,Savings or personal savings': 'Salary,Loan,Savings',
                  'Family support (provided by parents or other family members),Scholarships or financial aid': 'Family support,Scholarship',
                  'Income from work (part-time or full-time),Savings or personal savings': 'Salary,Savings',
                  'Family support (provided by parents or other family members),Scholarships or financial aid,Savings or personal savings': 'Family support,Scholarship,Savings',
                  'Family support (provided by parents or other family members),Scholarships or financial aid,Income from work (part-time or full-time),Savings or personal savings': 'Family support,Scholarship,Salary,Savings',
                  'Student loans': 'Loan',
                  'Family support (provided by parents or other family members),Income from work (part-time or full-time),Student loans': 'Family support,Salary,Loan',
                  'Family support (provided by parents or other family members),Scholarships or financial aid,Income from work (part-time or full-time),Student loans,Savings or personal savings': 'Family support,Scholarship,Salary,Loan,Savings',
                  'Savings or personal savings': 'Savings',
                  'Family support (provided by parents or other family members),Student loans': 'Family support,Loan',
                  'Scholarships or financial aid': 'Scholarship',
                  'Income from work (part-time or full-time),Student loans': 'Salary,Loan' 
                   }
DEMO_dataset.loc[:, "INCOME_SOURCE"] = DEMO_dataset["INCOME_SOURCE"].replace(INCOME_SOURCE_map)

#CategoricalDtype
categories_order_1 = ['Primary school', 'Secondary school', "Bachelor's degree", "Master's degree", "Doctor's degree"]
edu_dtype = CategoricalDtype(categories=categories_order_1, ordered=True)
DEMO_dataset['FATHER_EDU'] = DEMO_dataset['FATHER_EDU'].astype(edu_dtype)
DEMO_dataset['MOTHER_EDU'] = DEMO_dataset['MOTHER_EDU'].astype(edu_dtype)

categories_order_2 = ['Twins', 'Youngest son/daughter', 'Middle son/daughter', 'Oldest son/daughter']
birth_dtype = CategoricalDtype(categories=categories_order_2, ordered=True)
DEMO_dataset['BIRTH_ORDER'] = DEMO_dataset['BIRTH_ORDER'].astype(birth_dtype)

In [15]:
DEMO_dataset.dtypes

Duration(inseconds)     float64
Progress                  int64
ResponseId               object
GEN                      object
AGE                     float64
FACULTY                  object
RACE                     object
BIRTH_PLACE              object
HAVE_BELIEF              object
RELIGION                 object
MARR_STATUS              object
BIOKIDS                  object
HAVE_SIBLINGS            object
BIRTH_ORDER            category
WHO_SIBLINGS             object
BIO_SIBLINGS             object
CLOSE_FAMILY             object
CLOSE_FAMILY_NS          object
FATHER_EDU             category
MOTHER_EDU             category
INCOME_SOURCE            object
INCOME                  float64
dtype: object

In [16]:
FERT_dataset["SCALE_PLAN_FERT"] = pd.to_numeric(FERT_dataset["SCALE_PLAN_FERT"], errors="coerce")
FERT_dataset["BEST_FERT_AGE"] = pd.to_numeric(FERT_dataset["BEST_FERT_AGE"], errors="coerce")

#Mapping of answers
R_MARR_STATUS_map = {'單身': 'Single', '戀愛中': 'In a relationship', '不願透露': 'Prefer not to say', '訂婚/已婚': 'Engaged/Married',
                     '<label data-runtime-class-q-checked="runtime.Choices.1.Selected" for="QR~QID33~1" id="QID33-1-label">Single': 'Single',
                     '<label data-runtime-class-q-checked="runtime.Choices.2.Selected" for="QR~QID33~2" id="QID33-2-label">In a relationship': 'In a relationship'}
FERT_dataset.loc[:, "R_STATUS"] = FERT_dataset["R_STATUS"].replace(R_MARR_STATUS_map)

PLAN_FERT_map = {
    '也許或也許不': 'I am not sure', '可能是': 'Maybe yes', '肯定不': 'Absolutely not', '不願透露': 'Prefer not to say', '肯定是': 'Yes, definitely'
}
FERT_dataset.loc[:, "PLAN_FERT"] = FERT_dataset["PLAN_FERT"].replace(PLAN_FERT_map)

PARTNER_PLAN_FERT_map = {
    '可能是': 'Maybe yes', '肯定是': 'Yes, definitely', '也許或也許不': 'I am not sure'
}
FERT_dataset.loc[:, "PARTNER_PLAN_FERT"] = FERT_dataset["PARTNER_PLAN_FERT"].replace(PARTNER_PLAN_FERT_map)

SCALE_PLAN_FERT_map = {
    '可能是': 'Maybe yes', '肯定是': 'Yes, definitely', '也許或也許不': 'I am not sure'
}
FERT_dataset.loc[:, "SCALE_PLAN_FERT"] = FERT_dataset["SCALE_PLAN_FERT"].replace(SCALE_PLAN_FERT_map)

CHILD_NUM_map = {
    '1個': '1', '2個': '2', '0個': '0'
}
FERT_dataset.loc[:, "CHILD_NUM"] = FERT_dataset["CHILD_NUM"].replace(CHILD_NUM_map)
FERT_dataset["CHILD_NUM"] = pd.to_numeric(FERT_dataset["CHILD_NUM"], errors="coerce")

DISCUSS_CHILD_map = {
    '很少討論': 'We rarely discuss it', '是，我們經常討論': 'Yes, we discuss it frequently'
}
FERT_dataset.loc[:, "DISCUSS_CHILD"] = FERT_dataset["DISCUSS_CHILD"].replace(DISCUSS_CHILD_map)

CHILD_GEN_map = {
    '是，偏好女孩': 'Yes, I prefer girl', '沒有特定偏好': 'No special preference', '不願透露': 'Prefer not to say', '一男一女': 'No special preference'
}
FERT_dataset.loc[:, "CHILD_GEN"] = FERT_dataset["CHILD_GEN"].replace(CHILD_GEN_map)

FACTOR_FERT_PLAN_map = {
    '經濟狀況,伴侶或家庭支持,職業發展,健康狀況,生活品質,教育及撫養成本,政策及社會保障（如育兒補貼、教育政策等）,個人生活目標或價值觀,子女性別或數量偏好': 'Financial situation,Support from family or partner,Career and work development,Life quality,Cost of education and parenting,Policies and social security (such as childcare subsidies, education policies, etc.),Personal life goals or values,Preference for the gender or number of children',
       '經濟狀況,伴侶或家庭支持,職業發展,健康狀況,生活品質,教育及撫養成本,政策及社會保障（如育兒補貼、教育政策等）,個人生活目標或價值觀': 'Financial situation,Support from family or partner,Career and work development,Health state,Life quality,Cost of education and parenting,Policies and social security (such as childcare subsidies, education policies, etc.),Personal life goals or values',
       '經濟狀況,伴侶或家庭支持,健康狀況,生活品質,教育及撫養成本': 'Financial situation,Support from family or partner,Health state,Life quality,Cost of education and parenting', 
       '經濟狀況,職業發展,健康狀況,教育及撫養成本': 'Financial situation,Career and work development,Health state,Cost of education and parenting',
       '經濟狀況,職業發展,健康狀況,生活品質,教育及撫養成本': 'Financial situation,Career and work development,Health state,Life quality,Cost of education and parenting', 
       '經濟狀況': 'Financial situation', '個人生活目標或價值觀': 'Personal life goals or values',
       '經濟狀況,伴侶或家庭支持,職業發展,健康狀況,生活品質,教育及撫養成本': 'Financial situation,Support from family or partner,Career and work development,Health state,Life quality,Cost of education and parenting', 
       '伴侶或家庭支持,健康狀況,生活品質': 'Support from family or partner,Health state,Life quality',
       '教育及撫養成本,個人生活目標或價值觀': 'Cost of education and parenting,Personal life goals or values', 
       '政策及社會保障（如育兒補貼、教育政策等）': 'Policies and social security (such as childcare subsidies, education policies, etc.)',
       '經濟狀況,伴侶或家庭支持,職業發展,健康狀況,生活品質,教育及撫養成本,個人生活目標或價值觀': 'Financial situation,Support from family or partner,Career and work development,Health state,Life quality,Cost of education and parenting,Personal life goals or values', 
       '其他（請在下方欄註明）': np.nan,
       '經濟狀況,健康狀況,教育及撫養成本':'Financial situation,Health state,Cost of education and parenting',
       '經濟狀況,伴侶或家庭支持,職業發展,生活品質,個人生活目標或價值觀': 'Financial situation,Support from family or partner,Career and work development,Life quality,Personal life goals or values',
       '經濟狀況,職業發展,個人生活目標或價值觀': 'Financial situation,Career and work development,Personal life goals or values', 
       '經濟狀況,伴侶或家庭支持,職業發展,健康狀況,教育及撫養成本': 'Financial situation,Support from family or partner,Career and work development,Health state,Cost of education and parenting',
       '經濟狀況,職業發展,健康狀況': 'Financial situation,Career and work development,Health state', 
       '經濟狀況,伴侶或家庭支持,職業發展,健康狀況':'Financial situation,Support from family or partner,Career and work development,Health state',
       '經濟狀況,伴侶或家庭支持,職業發展,健康狀況,生活品質,教育及撫養成本,政策及社會保障（如育兒補貼、教育政策等）': 'Financial situation,Support from family or partner,Career and work development,Health state,Life quality,Cost of education and parenting,Policies and social security (such as childcare subsidies, education policies, etc.)'
}
FERT_dataset.loc[:, "FACTOR_FERT_PLAN"] = FERT_dataset["FACTOR_FERT_PLAN"].replace(FACTOR_FERT_PLAN_map)

FERT_PURPOSE_map = {
    '生育不應關於目的': 'It is not about goal', 
    '延續家庭血脈,增加家庭成員，豐富家庭生活,為晚年生活提供保障': 'Continue the family line,Increase family members and enrich family life,Provide security for old age',
    '體驗為人父母的生活,增加家庭成員，豐富家庭生活': 'Experience parenthood,Increase family members and enrich family life', 
    '增加家庭成員，豐富家庭生活,為晚年生活提供保障,配合伴侶的意願': "Experience parenthood,Increase family members and enrich family life,Provide security for old age,Align with the partner's wishes",
    '延續家庭血脈,增加家庭成員，豐富家庭生活,為晚年生活提供保障,滿足社會或家庭的期望': 'Continue the family line,Increase family members and enrich family life,Provide security for old age,Meet social or family expectations', 
    '為晚年生活提供保障': 'Provide security for old age',
    '生育不應關於目的,體驗為人父母的生活,增加家庭成員，豐富家庭生活,配合伴侶的意願,滿足社會或家庭的期望': "It is not about goal,Experience parenthood,Increase family members and enrich family life,Align with the partner's wishes,Meet social or family expectations",
    '體驗為人父母的生活,增加家庭成員，豐富家庭生活,個人成就感或人生使命': 'Experience parenthood,Increase family members and enrich family life,Personal fulfillment or life mission', 
    '個人成就感或人生使命':'Personal fulfillment or life mission', 
    '其他（請在下方欄註明）': np.nan,
    '生育不應關於目的,增加家庭成員，豐富家庭生活,為晚年生活提供保障': 'It is not about goal,Increase family members and enrich family life,Provide security for old age', 
    '滿足社會或家庭的期望': 'Meet social or family expectations',
    '延續家庭血脈,增加家庭成員，豐富家庭生活':'Continue the family line,Increase family members and enrich family life',
    '喜欢小孩子': 'Experience parenthood', 'Love': "Align with the partner's wishes"
}
FERT_dataset.loc[:, "FERT_PURPOSE"] = FERT_dataset["FERT_PURPOSE"].replace(FERT_PURPOSE_map)

FERT_VALUE_map = {
    '一般': 'Average', '比較重要': 'Relatively important', '不太重要': 'Not very important', '非常重要': 'Very important', '沒有價值':'No value'}
FERT_dataset.loc[:, "FERT_VALUE"] = FERT_dataset["FERT_VALUE"].replace(FERT_VALUE_map)

FERT_CAREER_map = {
    '有消極影響': 'Negative impact', '不確定': 'Not sure', '沒有影響': 'No impact', '有積極影響': 'Positive impact'}
FERT_dataset.loc[:, "FERT_CAREER"] = FERT_dataset["FERT_CAREER"].replace(FERT_CAREER_map)

WHO_PARENT_map = {
    '父母共同分擔': 'Both parents', '主力由母親負責': 'Primarily the mother', '保姆或其他專業人士': 'Nannies or other professionals'}
FERT_dataset.loc[:, "WHO_PARENT"] = FERT_dataset["WHO_PARENT"].replace(WHO_PARENT_map)

CHILD_NUM_INFLUENCE_map = {
    '可能有消極影響': 'May have a negative impact', 
    '有積極影響，但子女人數適中為佳': 'Positive impact, but an moderate number. of children is the best',
    '有積極影響，子女越多越幸福': 'Positive impact; the more children, the happier the family', 
    '沒有影響': 'No impact', '不確定': 'Not sure'}
FERT_dataset.loc[:, "CHILD_NUM_INFLUENCE"] = FERT_dataset["CHILD_NUM_INFLUENCE"].replace(CHILD_NUM_INFLUENCE_map)

PARENT_RESPONS_map = {
       '提供良好的生活和教育環境,支持子女個性和興趣的發展,為子女樹立道德榜樣,為子女創造經濟保障,鼓勵子女的自立和自信': "Provide good living and educational environment,Support the development of children's personalities and interests,Become a moral example for children,Create economic security for children,Encourage children's independence and self-confidence",
       '提供良好的生活和教育環境,支持子女個性和興趣的發展,為子女樹立道德榜樣,為子女創造經濟保障':"Provide good living and educational environment,Support the development of children's personalities and interests,Become a moral example for children,Create economic security for children",
       '提供良好的生活和教育環境': 'Provide good living and educational environment', 
       '提供良好的生活和教育環境,支持子女個性和興趣的發展,為子女創造經濟保障': "Provide good living and educational environment,Support the development of children's personalities and interests,Create economic security for children", 
       '為子女創造經濟保障': "Create economic security for children",
       '其他（請在下方欄註明）': np.nan, 
       '提供良好的生活和教育環境,為子女樹立道德榜樣,鼓勵子女的自立和自信': "Provide good living and educational environment,Become a moral example for children,Encourage children's independence and self-confidence",
       '提供良好的生活和教育環境,支持子女個性和興趣的發展': "Provide good living and educational environment,Support the development of children's personalities and interests"
}
FERT_dataset.loc[:, "PARENT_RESPONS"] = FERT_dataset["PARENT_RESPONS"].replace(PARENT_RESPONS_map)

FACTOR_FERT_map = {
       '自身經濟狀況,伴侶和家庭的支持程度,工作穩定性和職業前景,健康狀況,子女的教育和成長環境': 'Personal financial situation,Support level from partner and family,Job stability and career prospects,Health status,Education and growth environment for children',
       '自身經濟狀況,工作穩定性和職業前景,健康狀況,子女的教育和成長環境': 'Personal financial situation,Job stability and career prospects,Health status,Education and growth environment for children',
       '自身經濟狀況,伴侶和家庭的支持程度,工作穩定性和職業前景,健康狀況': 'Personal financial situation,Support level from partner and family,Job stability and career prospects,Health status', 
       '自身經濟狀況': 'Personal financial situation',
       '自身經濟狀況,工作穩定性和職業前景,健康狀況': 'Personal financial situation,Job stability and career prospects,Health status', 
       '自身經濟狀況,健康狀況': 'Personal financial situation,Health status', 
       '伴侶和家庭的支持程度,健康狀況': 'Support level from partner and family,Health status',
       '自身經濟狀況,工作穩定性和職業前景,子女的教育和成長環境': 'Personal financial situation,Job stability and career prospects,Education and growth environment for children',
       '自身經濟狀況,伴侶和家庭的支持程度': 'Personal financial situation,Support level from partner and family', 
       '子女的教育和成長環境': 'Education and growth environment for children',
       '自身經濟狀況,伴侶和家庭的支持程度,工作穩定性和職業前景': 'Personal financial situation,Support level from partner and family,Job stability and career prospects'
}
FERT_dataset.loc[:, "FACTOR_FERT"] = FERT_dataset["FACTOR_FERT"].replace(FACTOR_FERT_map)

#CategoricalDtype
categories_order_3 = ['Absolutely not', 'Maybe not', 'I am not sure', 'Maybe yes', 'Yes, definitely', 'Prefer not to say']
plan_fert_dtype = CategoricalDtype(categories=categories_order_3, ordered=True)
FERT_dataset['PLAN_FERT'] = FERT_dataset['PLAN_FERT'].astype(plan_fert_dtype)

categories_order_4 = ['We never discuss it', 'We rarely discuss it', 'Occasionally, we discuss it', 'Yes, we discuss it frequently']
discuss_dtype = CategoricalDtype(categories=categories_order_4, ordered=True)
FERT_dataset['DISCUSS_CHILD'] = FERT_dataset['DISCUSS_CHILD'].astype(discuss_dtype)

categories_order_5 = ['No value', 'Not very important', 'Average', 'Relatively important', 'Very important']
value_dtype = CategoricalDtype(categories=categories_order_5, ordered=True)
FERT_dataset['FERT_VALUE'] = FERT_dataset['FERT_VALUE'].astype(value_dtype)

FERT_dataset.dtypes

Duration(inseconds)     float64
Progress                  int64
ResponseId               object
R_STATUS                 object
PLAN_FERT              category
PARTNER_PLAN_FERT        object
SCALE_PLAN_FERT         float64
CHILD_NUM               float64
DISCUSS_CHILD          category
BEST_FERT_AGE           float64
CHILD_GEN                object
FACTOR_FERT_PLAN         object
FERT_PURPOSE             object
FERT_VALUE             category
FERT_CAREER              object
WHO_PARENT               object
CHILD_NUM_INFLUENCE      object
PARENT_RESPONS           object
FACTOR_FERT              object
dtype: object

In [17]:
MARR_dataset["ECO"] = pd.to_numeric(MARR_dataset["ECO"], errors="coerce")
MARR_dataset["COM"] = pd.to_numeric(MARR_dataset["COM"], errors="coerce")
MARR_dataset["PER"] = pd.to_numeric(MARR_dataset["PER"], errors="coerce")
MARR_dataset["EMO"] = pd.to_numeric(MARR_dataset["EMO"], errors="coerce")
MARR_dataset["INT"] = pd.to_numeric(MARR_dataset["INT"], errors="coerce")
MARR_dataset["LIF"] = pd.to_numeric(MARR_dataset["LIF"], errors="coerce")

#Mapping of answers
MAR_AGE_map = {'Other (please specify below)': np.nan}
MARR_dataset.loc[:, "MAR_AGE"] = MARR_dataset["MAR_AGE"].replace(MAR_AGE_map)

BEFORE_SEX_map = {'Other (please specify below)': np.nan}
MARR_dataset.loc[:, "BEFORE_SEX"] = MARR_dataset["BEFORE_SEX"].replace(BEFORE_SEX_map)

PARTNER_BEFORE_SEX_map = {'需要瞭解其motivation': np.nan, '独身主义，未考虑过': np.nan}
MARR_dataset.loc[:, "PARTNER_BEFORE_SEX"] = MARR_dataset["PARTNER_BEFORE_SEX"].replace(PARTNER_BEFORE_SEX_map)

R_BEFORE_SEX_map = {'沒有伴侶': np.nan, '独身主义，未考虑过': np.nan}
MARR_dataset.loc[:, "R_BEFORE_SEX"] = MARR_dataset["R_BEFORE_SEX"].replace(R_BEFORE_SEX_map)

ARRANGED_MAR_map = {'只能对于结果来评判，对于那些接受包办婚姻后自己认为过得幸福的就是好的，而对于那些婚后生活中都感到痛苦的就是不应该的。': "Neutral - I do not have a clear opinion on arranged marriages; I believe that everyone's circumstances are different and individual choices should be respected" }
MARR_dataset.loc[:, "ARRANGED_MAR"] = MARR_dataset["ARRANGED_MAR"].replace(ARRANGED_MAR_map)

EXCPET_MAR_map = {'沒有，因我認為沒有法律上效力，分開時難有保障': np.nan}
MARR_dataset.loc[:, "EXCPET_MAR"] = MARR_dataset["EXCPET_MAR"].replace(EXCPET_MAR_map)

MARR_dataset.dtypes

Duration(inseconds)    float64
Progress                 int64
ResponseId              object
R_STATUS                object
WILLING_MAR             object
MAR_AGE                 object
MAR_CHOICE              object
MAR_IMPORTANCE          object
LOVE_IN_MAR             object
FIN_IN_MAR              object
TRAD_IN_MAR             object
COR_IN_MRA              object
FULFIL_IN_MRA           object
ECO                    float64
COM                    float64
PER                    float64
EMO                    float64
INT                    float64
LIF                    float64
BEFORE_SEX              object
PARTNER_BEFORE_SEX      object
R_BEFORE_SEX            object
ARRANGED_MAR            object
EXCPET_MAR              object
dtype: object

In [18]:
PURPOSE_map = {
    '以前现在未来都没有这方面的打算': np.nan,
    'no': 'Having no specific purpose yet',
    '做爱': 'Obtaining happiness and fun in life',
    '没有这个必要': np.nan,
    'being a relationship is an exhausting thing in my opinion make a booty call may be better': 'Obtaining happiness and fun in life'
}
LOVE_dataset.loc[:, "PURPOSE"] = LOVE_dataset["PURPOSE"].replace(PURPOSE_map)

REASON_map = {
    '我們是哥布林 而且是不會爆金幣的哥布林': 'Economic pressure or limited resources',
    '擔心談戀愛影響自己的情緒和狀態，不想改變現狀': 'Fear of emotional setbacks or having concerns about dating',
    '没有这个必要': np.nan,
    'being a relationship is an exhausting thing in my opinion make a booty call may be better': 'Fear of emotional setbacks or having concerns about dating'
}
LOVE_dataset.loc[:, "REASON"] = LOVE_dataset["REASON"].replace(REASON_map)

CRITERIA_map = {'无': np.nan, '感觉': 'No fixed standards'}
LOVE_dataset.loc[:, "CRITERIA"] = LOVE_dataset["CRITERIA"].replace(CRITERIA_map)

VIEW_map = {'非常反对': 'Opposed', '富人親身感受  窮人親眼目睹': np.nan, 
           '感觉大部分人的大学恋爱只是因为大学之前没有接触过恋爱所以为了谈恋爱而谈恋爱，大部分校园情侣不会考虑毕业之后两人的发展方向，即面临毕业即分手的情况。': 'Opposed, believing that it is not advisable to invest emotionally during the school period'}
LOVE_dataset.loc[:, "VIEW"] = LOVE_dataset["VIEW"].replace(VIEW_map)

EXPECTATION_SINGLE_map = {
    '无': np.nan,
    'Other (please specify below)': np.nan, '無伴侶':np.nan
}
LOVE_dataset.loc[:, "EXPECTATION_SINGLE"] = LOVE_dataset["EXPECTATION_SINGLE"].replace(EXPECTATION_SINGLE_map)

#CategoricalDtype
categories_order_1 = ['Single', 'In a relationship', 'Engaged/Married', 'Divorced','Prefer not to say','nan']
status_dtype = CategoricalDtype(categories=categories_order_1, ordered=True)
LOVE_dataset['R_STATUS'] = LOVE_dataset['R_STATUS'].astype(status_dtype)


categories_order_2 = ['No romantic experience', 'Once', 'Twice','Three times or more','Prefer not to say']
times_dtype = CategoricalDtype(categories=categories_order_2, ordered=True)
LOVE_dataset['TIMES'] = LOVE_dataset['TIMES'].astype(times_dtype)

categories_order_3 = ['Before 18', '18-22', '23-27','Above 27']
idea_age_dtype = CategoricalDtype(categories=categories_order_3, ordered=True)
LOVE_dataset['IDEA_AGE'] = LOVE_dataset['IDEA_AGE'].astype(idea_age_dtype)

categories_order_4 = ['Before 15', '16-18', '19-21','22-25','Above 26','Prefer not to say']
first_age_dtype = CategoricalDtype(categories=categories_order_4, ordered=True)
LOVE_dataset['FIRST_AGE'] = LOVE_dataset['FIRST_AGE'].astype(first_age_dtype)

LOVE_dataset.dtypes

Duration(inseconds)     float64
Progress                  int64
ResponseId               object
R_STATUS               category
TIMES                  category
IDEA_AGE               category
FIRST_AGE              category
PURPOSE                  object
REASON                   object
CRITERIA                 object
VIEW                     object
EXPECTATION_SINGLE       object
EXPECTATION              object
AFFECT                   object
EVALUATE_MAN             object
EVALUATE_BESELF          object
EVALUATE_NEGOTIATE       object
EVALUATE_EMO             object
EVALUATE_HOPE            object
dtype: object

In [19]:
CULT_MARR_ATT_map = {
    '比較重要，受到多數人的重視': 'Relatively important, valued by most people', 
    '一般重要，婚姻重要性因人而異': 'Moderately important, the importance of marriage varies from person to person',
    '非常重要，是生活中不可或缺的一部分': 'Very important, an indispensable part of life',
    '完全不重要，婚姻在文化中不被強調': 'Not important at all, marriage is not emphasized in the culture'
}
CULT_dataset.loc[:, 'CULT_MARR_ATT'] = CULT_dataset['CULT_MARR_ATT'].replace(CULT_MARR_ATT_map)

FAMILY_IMPACT_map = {
    '家庭和社區對我的婚姻及生育決定有較大影響': 'Family and community have a significant influence on my marriage and fertility decisions', 
    '家庭和社區有一定影響，但我主要依據自己的意願': 'Family and community have some influence, but I mainly base my decisions on my own preferences',
       '家庭和社區影響較小，我更依靠個人選擇': 'Family and community have little influence, I rely more on personal choice', 
    '家庭和社區對我的婚姻及生育決定有決定性影響': 'Family and community have a decisive influence on my marriage and fertility decisions',
       '家庭和社區完全不影響我的婚姻及生育決定': 'Family and community have no influence on my marriage and fertility decisions'
}
CULT_dataset.loc[:, 'FAMILY_IMPACT'] = CULT_dataset['FAMILY_IMPACT'].replace(FAMILY_IMPACT_map)

CULT_FERT_ATT_map = {
    '不太重要，生育逐漸被視為個人選擇': 'Not very important, having children is increasingly seen as a personal choice', 
    '比較重要，多數人認為生育有其必要性': 'Relatively important, most people consider having children necessary', 
    '一般重要，生育重要性因人而異': 'Moderately important, the importance of having children varies from person to person',
       '非常重要，是人生中不可或缺的一部分': 'Very important, an indispensable part of life', 
    '完全不重要，生育在文化中並未被強調': 'Not important at all, having children is not emphasized in the culture'
}
CULT_dataset.loc[:, 'CULT_FERT_ATT'] = CULT_dataset['CULT_FERT_ATT'].replace(CULT_FERT_ATT_map)

WEDDING_map = {
    '比較重要，對婚姻有較大意義': 'Very important, an indispensable part of marriage', 
    '非常重要，是婚姻中不可或缺的一部分': 'Very important, an indispensable part of marriage', 
    '一般重要，是否舉行儀式取決於個人選擇': 'Moderately important, whether to hold a ceremony depends on personal choice',
       '不太重要，儀式並非婚姻的必要組成部分': 'Not very important, rituals are not a necessary component of marriage', 
    '完全不重要，婚姻不依賴文化儀式':'Not important at all, marriage does not rely on cultural rituals'
}
CULT_dataset.loc[:, 'WEDDING'] = CULT_dataset['WEDDING'].replace(WEDDING_map)

CULT_map = {
    '某種程度上反對': 'Somewhat disagree',
       '某種程度上同意': 'Somewhat agree', 
    '既不同意也不反對': 'Neither agree or disagree', '非常同意': 'Strongly agree', '強烈反對': 'Strongly disagree'
}
CULT_dataset.loc[:, 'FAMILY_BOOK'] = CULT_dataset['FAMILY_BOOK'].replace(CULT_map)
CULT_dataset.loc[:, 'FAMILY_RESOURCE'] = CULT_dataset['FAMILY_RESOURCE'].replace(CULT_map)
CULT_dataset.loc[:, 'PARICIPATION'] = CULT_dataset['PARICIPATION'].replace(CULT_map)
CULT_dataset.loc[:, 'STUDY_HABIT'] = CULT_dataset['STUDY_HABIT'].replace(CULT_map)
CULT_dataset.loc[:, 'MEDIA'] = CULT_dataset['MEDIA'].replace(CULT_map)
CULT_dataset.loc[:, 'SOCIAL'] = CULT_dataset['SOCIAL'].replace(CULT_map)
CULT_dataset.loc[:, 'TOPIC'] = CULT_dataset['TOPIC'].replace(CULT_map)
CULT_dataset.loc[:, 'DIVERSITY'] = CULT_dataset['DIVERSITY'].replace(CULT_map)

CULT_dataset.dtypes

Duration(inseconds)    float64
Progress                 int64
ResponseId              object
CULT_MARR_ATT           object
CULT_FERT_ATT           object
FAMILY_IMPACT           object
WEDDING                 object
FAMILY_BOOK             object
FAMILY_RESOURCE         object
PARICIPATION            object
STUDY_HABIT             object
MEDIA                   object
SOCIAL                  object
TOPIC                   object
DIVERSITY               object
dtype: object

In [20]:
RCFE_dataset.dtypes

Duration(inseconds)    float64
Progress                 int64
ResponseId              object
FAM_EDU                 object
PAR_MARR                object
CHILDHOOD               object
TRAD_IMPORTANCE         object
IDENTITY                object
CULT_CONFIDENCE         object
FAMILY_INFLUENCE        object
FAM_ADVICE              object
ON_PATENER_ADVICE       object
TRAD_MAR                object
PRIORITY                object
dtype: object

In [21]:
GDRO_dataset.dtypes

Duration(inseconds)    float64
Progress                 int64
ResponseId              object
PAR_EVEN                object
ECO_MALE                object
DECISION_MALE           object
PAR_BOTH                object
LABOUR                  object
MALE_LABOUR             object
WORKING_GDRO            object
ABS_EQUAL               object
FE_ALL                  object
PREJ                    object
dtype: object

In [22]:
DEMO_ID = DEMO_dataset['ResponseId'].tolist()
LOVE_ID = LOVE_dataset['ResponseId'].tolist()
MARR_ID = MARR_dataset['ResponseId'].tolist()
FERT_ID = FERT_dataset['ResponseId'].tolist()
CULT_ID = CULT_dataset['ResponseId'].tolist()
RCFE_ID = RCFE_dataset['ResponseId'].tolist()
GDRO_ID = GDRO_dataset['ResponseId'].tolist()

All = list(set(DEMO_ID + LOVE_ID + MARR_ID + FERT_ID + CULT_ID + RCFE_ID + GDRO_ID))
len(All) == len(DEMO_dataset) #No duplicated ID value

True

In [23]:
datasets = [DEMO_dataset, LOVE_dataset, MARR_dataset, FERT_dataset, CULT_dataset, RCFE_dataset, GDRO_dataset]
datasets_indexed = [df.set_index('ResponseId') for df in datasets]
merged_df = pd.concat(datasets_indexed, axis=1, join='outer')
merged_df.reset_index(inplace=True)
merged_df

,ResponseId,Duration(inseconds),Progress,GEN,AGE,FACULTY,RACE,BIRTH_PLACE,HAVE_BELIEF,RELIGION,MARR_STATUS,BIOKIDS,HAVE_SIBLINGS,BIRTH_ORDER,WHO_SIBLINGS,BIO_SIBLINGS,CLOSE_FAMILY,CLOSE_FAMILY_NS,FATHER_EDU,MOTHER_EDU,INCOME_SOURCE,INCOME,Duration(inseconds),Progress,R_STATUS,TIMES,IDEA_AGE,FIRST_AGE,PURPOSE,REASON,CRITERIA,VIEW,EXPECTATION_SINGLE,EXPECTATION,AFFECT,EVALUATE_MAN,EVALUATE_BESELF,EVALUATE_NEGOTIATE,EVALUATE_EMO,EVALUATE_HOPE,Duration(inseconds),Progress,R_STATUS,WILLING_MAR,MAR_AGE,MAR_CHOICE,MAR_IMPORTANCE,LOVE_IN_MAR,FIN_IN_MAR,TRAD_IN_MAR,COR_IN_MRA,FULFIL_IN_MRA,ECO,COM,PER,EMO,INT,LIF,BEFORE_SEX,PARTNER_BEFORE_SEX,R_BEFORE_SEX,ARRANGED_MAR,EXCPET_MAR,Duration(inseconds),Progress,R_STATUS,PLAN_FERT,PARTNER_PLAN_FERT,SCALE_PLAN_FERT,CHILD_NUM,DISCUSS_CHILD,BEST_FERT_AGE,CHILD_GEN,FACTOR_FERT_PLAN,FERT_PURPOSE,FERT_VALUE,FERT_CAREER,WHO_PARENT,CHILD_NUM_INFLUENCE,PARENT_RESPONS,FACTOR_FERT,Duration(inseconds),Progress,CULT_MARR_ATT,CULT_FERT_ATT,FAMILY_IMPACT,WEDDING,FAMILY_BOOK,FAMILY_RESOURCE,PARICIPATION,STUDY_HABIT,MEDIA,SOCIAL,TOPIC,DIVERSITY,Duration(inseconds),Progress,FAM_EDU,PAR_MARR,CHILDHOOD,TRAD_IMPORTANCE,IDENTITY,CULT_CONFIDENCE,FAMILY_INFLUENCE,FAM_ADVICE,ON_PATENER_ADVICE,TRAD_MAR,PRIORITY,Duration(inseconds),Progress,PAR_EVEN,ECO_MALE,DECISION_MALE,PAR_BOTH,LABOUR,MALE_LABOUR,WORKING_GDRO,ABS_EQUAL,FE_ALL,PREJ
0,R_4MM1ltT1Nhg82pr,2033.0,100,Male,20.0,FAH,Chinese,Mainland China,No,NaN,Parents are married,Born as a biological child,No siblings,NaN,NaN,NaN,NaN,"Father,Mother",Master's degree,Bachelor's degree,Family support,2275.0,2033.0,100.0,Single,Once,18-22,19-21,"Seeking emotional support and companionship,Pursuing marriage and long-term relationships,Personal growth and self-exploration,Obtaining happiness and fun in life",Fear of emotional setbacks or having concerns about dating,most important: that person must be positive and optimistic!,"Supportive, thinking that it can enrich the college experience","if that person is good enough I would want to get married with her ,or I will focused on now",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2033.0,100.0,"Very important, an indispensable part of life","Relatively important, most people consider having children necessary","Family and community have some influence, but I mainly base my decisions on my own preferences","Very important, an indispensable part of marriage",Strongly agree,Somewhat agree,Somewhat agree,Somewhat agree,Neither agree or disagree,Somewhat agree,Somewhat agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R_4X4Yrj2WHp3iDX9,292.0,100,Female,21.0,FHS,Chinese,Mainland China,No,NaN,Parents are married,Born as a biological child,No siblings,NaN,NaN,NaN,NaN,Mother,Bachelor's degree,NaN,Family support,4746.0,292.0,100.0,In a relationship,Three times or more,18-22,Before 15,"Seeking emotional support and companionship,Obtaining happiness and fun in life","Focusing on academics or career development,Economic pressure or limited resources,Lack of suitable partners","Compatibility of personality and worldview,Physical appearance and attractiveness,Compatibility in lifestyle and hobbies","Neutral, believing that whether to engage in a romantic relationship depends on personal choice",NaN,"The future is uncertain, and we are letting things develop naturally","There might be some impact, but I would actively cope with it and adjust my mindset​​​​​",Somewhat agree,Strongly agree,Neither agree or disagree,Neither agree or disagree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292.0,100.0,"Relatively important, valued by most people","Relatively important, most people consider having children

In [24]:
merged_df.dtypes

ResponseId               object
Duration(inseconds)     float64
Progress                  int64
GEN                      object
AGE                     float64
FACULTY                  object
RACE                     object
BIRTH_PLACE              object
HAVE_BELIEF              object
RELIGION                 object
MARR_STATUS              object
BIOKIDS                  object
HAVE_SIBLINGS            object
BIRTH_ORDER            category
WHO_SIBLINGS             object
BIO_SIBLINGS             object
CLOSE_FAMILY             object
CLOSE_FAMILY_NS          object
FATHER_EDU             category
MOTHER_EDU             category
INCOME_SOURCE            object
INCOME                  float64
Duration(inseconds)     float64
Progress                float64
R_STATUS               category
TIMES                  category
IDEA_AGE               category
FIRST_AGE              category
PURPOSE                  object
REASON                   object
CRITERIA                 object
VIEW    

In [25]:
AFFECT_map = {
   'There might be some impact, but I would actively cope with it and adjust my mindset\u200b\u200b\u200b\u200b\u200b':2,
       'There would be no impact, and I could quickly return to my normal life and study routine':4,
       'I would be greatly affected and need time to adjust my emotions':1,
       'The impact would be limited and have no long-term effect on my studies or life':3,
       'Prefer not to say':0
}
merged_df.loc[:, 'AFFECT'] = merged_df['AFFECT'].replace(AFFECT_map)
merged_df["AFFECT"] = pd.to_numeric(merged_df["AFFECT"], errors="coerce")

EVALUATE_MAN_map = {
   'Somewhat agree':4, 'Strongly agree':5, 'Somewhat disagree':2, 'Neither agree or disagree':3, 'Strongly disagree':1
}
merged_df.loc[:, 'EVALUATE_MAN'] = merged_df['EVALUATE_MAN'].replace(EVALUATE_MAN_map)
merged_df["EVALUATE_MAN"] = pd.to_numeric(merged_df["EVALUATE_MAN"], errors="coerce")
merged_df.loc[:, 'EVALUATE_MAN'] = merged_df['EVALUATE_MAN'].replace(EVALUATE_MAN_map)
merged_df["EVALUATE_MAN"] = pd.to_numeric(merged_df["EVALUATE_MAN"], errors="coerce")

EVALUATE_BESELF_map = {
   'Somewhat agree':4, 'Strongly agree':5, 'Somewhat disagree':2, 'Neither agree or disagree':3, 'Strongly disagree':1
}
merged_df.loc[:, 'EVALUATE_BESELF'] = merged_df['EVALUATE_BESELF'].replace(EVALUATE_BESELF_map)
merged_df["EVALUATE_BESELF"] = pd.to_numeric(merged_df["EVALUATE_BESELF"], errors="coerce")

EVALUATE_NEGOTIATE_map = {
   'Somewhat agree':4, 'Strongly agree':5, 'Somewhat disagree':2, 'Neither agree or disagree':3, 'Strongly disagree':1
}
merged_df.loc[:, 'EVALUATE_NEGOTIATE'] = merged_df['EVALUATE_NEGOTIATE'].replace(EVALUATE_NEGOTIATE_map)
merged_df["EVALUATE_NEGOTIATE"] = pd.to_numeric(merged_df["EVALUATE_NEGOTIATE"], errors="coerce")

EVALUATE_EMO_map = {
   'Somewhat agree':4, 'Strongly agree':5, 'Somewhat disagree':2, 'Neither agree or disagree':3, 'Strongly disagree':1
}
merged_df.loc[:, 'EVALUATE_EMO'] = merged_df['EVALUATE_EMO'].replace(EVALUATE_EMO_map)
merged_df["EVALUATE_EMO"] = pd.to_numeric(merged_df["EVALUATE_EMO"], errors="coerce")

EVALUATE_HOPE_map = {
   'Somewhat agree':4, 'Strongly agree':5, 'Somewhat disagree':2, 'Neither agree or disagree':3, 'Strongly disagree':1
}
merged_df.loc[:, 'EVALUATE_HOPE'] = merged_df['EVALUATE_HOPE'].replace(EVALUATE_HOPE_map)
merged_df["EVALUATE_HOPE"] = pd.to_numeric(merged_df["EVALUATE_HOPE"], errors="coerce")

WILLING_MAR_map = {
   'Yes, I am very willing':5,
       'I am not sure yet, I need to consider more factors':3,
       'Not very willing, I have a reserved attitude towards marriage':2,
       'Yes, but it depends on the specific circumstances':4,
       'Prefer not to say':0, 'Completely not, I prefer a non-marital life':1
}
merged_df.loc[:, 'WILLING_MAR'] = merged_df['WILLING_MAR'].replace(WILLING_MAR_map)
merged_df["WILLING_MAR"] = pd.to_numeric(merged_df["WILLING_MAR"], errors="coerce")

MAR_CHOICE_map = {
   'Yes, but more and more young people are leaning towards singleness':4,
       'Unlikely, more and more people are choosing to remain unmarried or delaying marriage':2,
       'Yes, I think most people will choose to get married':5,
       'Uncertain, it depends on individual values and cultural background':3,
       'No, most people (especially the young) tend to choose a non-marital lifestyle':1
}
merged_df.loc[:, 'MAR_CHOICE'] = merged_df['MAR_CHOICE'].replace(MAR_CHOICE_map)
merged_df["MAR_CHOICE"] = pd.to_numeric(merged_df["MAR_CHOICE"], errors="coerce")

MAR_IMPORTANCE_map = {
   'Relatively important, marriage can provide stability and support in life':4,
       'Moderately important, the importance of marriage varies from person to person':3,
       'Not very important, marriage is not a necessary condition for a happy life':2,
       'Not important at all, personal life can thrive without marriage':1,
       'Very important, marriage is an indispensable part of life':5
}
merged_df.loc[:, 'MAR_IMPORTANCE'] = merged_df['MAR_IMPORTANCE'].replace(MAR_IMPORTANCE_map)
merged_df["MAR_IMPORTANCE"] = pd.to_numeric(merged_df["MAR_IMPORTANCE"], errors="coerce")

MAR_IMPORTANCE_map = {
   'Relatively important, marriage can provide stability and support in life':4,
       'Moderately important, the importance of marriage varies from person to person':3,
       'Not very important, marriage is not a necessary condition for a happy life':2,
       'Not important at all, personal life can thrive without marriage':1,
       'Very important, marriage is an indispensable part of life':5
}
merged_df.loc[:, 'MAR_IMPORTANCE'] = merged_df['MAR_IMPORTANCE'].replace(MAR_IMPORTANCE_map)
merged_df["MAR_IMPORTANCE"] = pd.to_numeric(merged_df["MAR_IMPORTANCE"], errors="coerce")

LOVE_IN_MAR_map = {
   'Strongly agree':5, 'Somewhat agree':4, 'Somewhat disagree':2,
       'Strongly disagree':1, 'Neither disagree nor agree':3
}
merged_df.loc[:, 'LOVE_IN_MAR'] = merged_df['LOVE_IN_MAR'].replace(LOVE_IN_MAR_map)
merged_df["LOVE_IN_MAR"] = pd.to_numeric(merged_df["LOVE_IN_MAR"], errors="coerce")

FIN_IN_MAR_map = {
   'Strongly agree':5, 'Somewhat agree':4, 'Somewhat disagree':2,
       'Strongly disagree':1, 'Neither disagree nor agree':3
}
merged_df.loc[:, 'FIN_IN_MAR'] = merged_df['FIN_IN_MAR'].replace(FIN_IN_MAR_map)
merged_df["FIN_IN_MAR"] = pd.to_numeric(merged_df["FIN_IN_MAR"], errors="coerce")

TRAD_IN_MAR_map = {
   'Strongly agree':5, 'Somewhat agree':4, 'Somewhat disagree':2,
       'Strongly disagree':1, 'Neither disagree nor agree':3
}
merged_df.loc[:, 'TRAD_IN_MAR'] = merged_df['TRAD_IN_MAR'].replace(TRAD_IN_MAR_map)
merged_df["TRAD_IN_MAR"] = pd.to_numeric(merged_df["TRAD_IN_MAR"], errors="coerce")

COR_IN_MAR_map = {
   'Strongly agree':5, 'Somewhat agree':4, 'Somewhat disagree':2,
       'Strongly disagree':1, 'Neither disagree nor agree':3
}
merged_df.loc[:, 'COR_IN_MRA'] = merged_df['COR_IN_MRA'].replace(COR_IN_MAR_map)
merged_df["COR_IN_MRA"] = pd.to_numeric(merged_df["COR_IN_MRA"], errors="coerce")

FULFIL_IN_MRA_map = {
   'Strongly agree':5, 'Somewhat agree':4, 'Somewhat disagree':2,
       'Strongly disagree':1, 'Neither disagree nor agree':3
}
merged_df.loc[:, 'FULFIL_IN_MRA'] = merged_df['FULFIL_IN_MRA'].replace(FULFIL_IN_MRA_map)
merged_df["FULFIL_IN_MRA"] = pd.to_numeric(merged_df["FULFIL_IN_MRA"], errors="coerce")

BEFORE_SEX_map = {
   'Completely supportive - believing it is a normal personal choice':5,
       'Neutral - believing it is a personal choice and having no particular opinion':3,
       'Somewhat supportive - believing it can help enhance the relationship between partners':4,
       'Completely unsupportive - believing intimate relationships should only be established after marriage':1,
       'Somewhat unsupportive - believing it may have negative impacts on marriage':2
}
merged_df.loc[:, 'BEFORE_SEX'] = merged_df['BEFORE_SEX'].replace(BEFORE_SEX_map)
merged_df["BEFORE_SEX"] = pd.to_numeric(merged_df["BEFORE_SEX"], errors="coerce")

PARTNER_BEFORE_SEX_map = {
   "Somewhat supportive - as long as both parties are consenting, it won't affect my view of my partner":4,
       'Neutral - this is a personal privacy matter, and I won’t judge too much':3,
       "Completely supportive - it's a personal choice and I won’t be concerned about my partner's past":5,
       'Somewhat unsupportive - it might have some impact on future married life, but I will be understanding':2,
       'Completely unsupportive - this might conflict with my values and bring uncertainty to future marriage':1
}
merged_df.loc[:, 'PARTNER_BEFORE_SEX'] = merged_df['PARTNER_BEFORE_SEX'].replace(PARTNER_BEFORE_SEX_map)
merged_df["PARTNER_BEFORE_SEX"] = pd.to_numeric(merged_df["PARTNER_BEFORE_SEX"], errors="coerce")

R_BEFORE_SEX_map = {
   'Somewhat supportive - My partner might be understanding as long as our current relationship is based on mutual respect and trust':4,
       'Neutral - My partner might consider this my personal privacy and not comment much, focusing instead on our current relationship':3,
       'Somewhat unsupportive - My partner might express some concerns, worrying that this might affect our future marriage life':2,
       'Completely unsupportive - If my partner holds a more conservative viewpoint, they might express disapproval':1,
       'Completely supportive - My partner might see this as my personal past experience and not care or judge much':5
}
merged_df.loc[:, 'R_BEFORE_SEX'] = merged_df['R_BEFORE_SEX'].replace(R_BEFORE_SEX_map)
merged_df["R_BEFORE_SEX"] = pd.to_numeric(merged_df["R_BEFORE_SEX"], errors="coerce")

ARRANGED_MAR_map = {
    
       'Somewhat opposed - I believe that arranged marriages may limit personal freedom and affect happiness': 'Somewhat unsupportive - I believe that arranged marriages may limit personal freedom and affect happiness',
       'Completely opposed - I firmly believe that marriage should be based on personal choice and love; arranged marriages are unacceptable':'Completely unsupportive - I firmly believe that marriage should be based on personal choice and love; arranged marriages are unacceptable'
}     
merged_df.loc[:, 'ARRANGED_MAR'] = merged_df['ARRANGED_MAR'].replace(ARRANGED_MAR_map)

ARRANGED_MAR_map = {
   "Neutral - I do not have a clear opinion on arranged marriages; I believe that everyone's circumstances are different and individual choices should be respected":3,
       'Somewhat unsupportive - I believe that arranged marriages may limit personal freedom and affect happiness':2,
       'Completely unsupportive - I firmly believe that marriage should be based on personal choice and love; arranged marriages are unacceptable':1,
       'Completely supportive - I believe that arranged marriage is an effective form of marriage that can bring stability and happiness':5,
       'Somewhat supportive - I think arranged marriage can be a reasonable choice in certain situations, but individual willingness should also be considered.':4
}
merged_df.loc[:, 'ARRANGED_MAR'] = merged_df['ARRANGED_MAR'].replace(ARRANGED_MAR_map)
merged_df["ARRANGED_MAR"] = pd.to_numeric(merged_df["ARRANGED_MAR"], errors="coerce")

FERT_PLAN_map = {
   'I am not sure':3, 'Maybe yes':4, 'Absolutely not':1, 'Prefer not to say':0, 'Yes, definitely':5, 'Maybe not':2
}
merged_df.loc[:, 'PLAN_FERT'] = merged_df['PLAN_FERT'].replace(FERT_PLAN_map)
merged_df["PLAN_FERT"] = pd.to_numeric(merged_df["PLAN_FERT"], errors="coerce")


PARTNER_FERT_PLAN_map = {
   'Maybe yes':2, 'Yes, definitely':3, 'I am not sure':1
}
merged_df.loc[:, 'PARTNER_PLAN_FERT'] = merged_df['PARTNER_PLAN_FERT'].replace(PARTNER_PLAN_FERT_map)
merged_df["PARTNER_PLAN_FERT"] = pd.to_numeric(merged_df["PARTNER_PLAN_FERT"], errors="coerce")

DISCUSS_CHILD_map = {
   'We rarely discuss it':2, 'Yes, we discuss it frequently':4, 'We never discuss it':1, 'Occasionally, we discuss it':3
}
merged_df.loc[:, 'DISCUSS_CHILD'] = merged_df['DISCUSS_CHILD'].replace(DISCUSS_CHILD_map)
merged_df["DISCUSS_CHILD"] = pd.to_numeric(merged_df["DISCUSS_CHILD"], errors="coerce")

FERT_VALUE_map = {
   'Average':3, 'Relatively important':4, 'Not very important':2, 'Very important':5, 'No value':1
}
merged_df.loc[:, 'FERT_VALUE'] = merged_df['FERT_VALUE'].replace(FERT_VALUE_map)
merged_df["FERT_VALUE"] = pd.to_numeric(merged_df["FERT_VALUE"], errors="coerce")

CULT_MARR_ATT_map = {
   'Very important, an indispensable part of life':5,
       'Relatively important, valued by most people':4,
       'Moderately important, the importance of marriage varies from person to person':3,
       'Not very important, marriage is increasingly seen as a personal choice':2,
       'Not important at all, marriage is not emphasized in the culture':1
}
merged_df.loc[:, 'CULT_MARR_ATT'] = merged_df['CULT_MARR_ATT'].replace(CULT_MARR_ATT_map)
merged_df["CULT_MARR_ATT"] = pd.to_numeric(merged_df["CULT_MARR_ATT"], errors="coerce")

CULT_FERT_ATT_map = {
   'Relatively important, most people consider having children necessary':4,
       'Not very important, having children is increasingly seen as a personal choice':2,
       'Moderately important, the importance of having children varies from person to person':3,
       'Very important, an indispensable part of life':5,
       'Not important at all, having children is not emphasized in the culture':1
}
merged_df.loc[:, 'CULT_FERT_ATT'] = merged_df['CULT_FERT_ATT'].replace(CULT_FERT_ATT_map)
merged_df["CULT_FERT_ATT"] = pd.to_numeric(merged_df["CULT_FERT_ATT"], errors="coerce")

FAMILY_IMPACT_map = {
   'Family and community have some influence, but I mainly base my decisions on my own preferences':3,
       'Family and community have no influence on my marriage and fertility decisions':1,
       'Family and community have a decisive influence on my marriage and fertility decisions':5,
       'Prefer not to say':0,
       'Family and community have little influence, I rely more on personal choice':2,
       'Family and community have a significant influence on my marriage and fertility decisions':4
}
merged_df.loc[:, 'FAMILY_IMPACT'] = merged_df['FAMILY_IMPACT'].replace(FAMILY_IMPACT_map)
merged_df["FAMILY_IMPACT"] = pd.to_numeric(merged_df["FAMILY_IMPACT"], errors="coerce")

WEDDING_map = {
   'Very important, an indispensable part of marriage':5,
       'Moderately important, whether to hold a ceremony depends on personal choice':3,
       'Relatively important, holds significant meaning in marriage':4,
       'Not very important, rituals are not a necessary component of marriage':2,
       'Not important at all, marriage does not rely on cultural rituals':1
}
merged_df.loc[:, 'WEDDING'] = merged_df['WEDDING'].replace(WEDDING_map)
merged_df["WEDDING"] = pd.to_numeric(merged_df["WEDDING"], errors="coerce")

FAMILY_BOOK_map = {
   'Strongly agree':5, 'Somewhat agree':4, 
       'Neither agree or disagree':3, 'Strongly disagree':1,
       'Somewhat disagree':2
}
merged_df.loc[:, 'FAMILY_BOOK'] = merged_df['FAMILY_BOOK'].replace(FAMILY_BOOK_map)
merged_df["FAMILY_BOOK"] = pd.to_numeric(merged_df["FAMILY_BOOK"], errors="coerce")

FAMILY_RESOURCE_map = {
   'Strongly agree':5, 'Somewhat agree':4, 
       'Neither agree or disagree':3, 'Strongly disagree':1,
       'Somewhat disagree':2
}
merged_df.loc[:, 'FAMILY_RESOURCE'] = merged_df['FAMILY_RESOURCE'].replace(FAMILY_RESOURCE_map)
merged_df["FAMILY_RESOURCE"] = pd.to_numeric(merged_df["FAMILY_RESOURCE"], errors="coerce")

PARICIPATION_map  = {
   'Strongly agree':5, 'Somewhat agree':4, 
       'Neither agree or disagree':3, 'Strongly disagree':1,
       'Somewhat disagree':2
}
merged_df.loc[:, 'PARICIPATION'] = merged_df['PARICIPATION'].replace(PARICIPATION_map)
merged_df["PARICIPATION"] = pd.to_numeric(merged_df["PARICIPATION"], errors="coerce")

STUDY_HABIT_map  = {
   'Strongly agree':5, 'Somewhat agree':4, 
       'Neither agree or disagree':3, 'Strongly disagree':1,
       'Somewhat disagree':2
}
merged_df.loc[:, 'STUDY_HABIT'] = merged_df['STUDY_HABIT'].replace(STUDY_HABIT_map)
merged_df["STUDY_HABIT"] = pd.to_numeric(merged_df["STUDY_HABIT"], errors="coerce")

MEDIA_map  = {
   'Strongly agree':5, 'Somewhat agree':4, 
       'Neither agree or disagree':3, 'Strongly disagree':1,
       'Somewhat disagree':2
}
merged_df.loc[:, 'MEDIA'] = merged_df['MEDIA'].replace(MEDIA_map)
merged_df["MEDIA"] = pd.to_numeric(merged_df["MEDIA"], errors="coerce")

SOCIAL_map  = {
   'Strongly agree':5, 'Somewhat agree':4, 
       'Neither agree or disagree':3, 'Strongly disagree':1,
       'Somewhat disagree':2
}
merged_df.loc[:, 'SOCIAL'] = merged_df['SOCIAL'].replace(SOCIAL_map)
merged_df["SOCIAL"] = pd.to_numeric(merged_df["SOCIAL"], errors="coerce")

TOPIC_map  = {
   'Strongly agree':5, 'Somewhat agree':4, 
       'Neither agree or disagree':3, 'Strongly disagree':1,
       'Somewhat disagree':2
}
merged_df.loc[:, 'TOPIC'] = merged_df['TOPIC'].replace(TOPIC_map)
merged_df["TOPIC"] = pd.to_numeric(merged_df["TOPIC"], errors="coerce")

DIVERSITY_map  = {
   'Strongly agree':5, 'Somewhat agree':4, 
       'Neither agree or disagree':3, 'Strongly disagree':1,
       'Somewhat disagree':2
}
merged_df.loc[:, 'DIVERSITY'] = merged_df['DIVERSITY'].replace(DIVERSITY_map)
merged_df["DIVERSITY"] = pd.to_numeric(merged_df["DIVERSITY"], errors="coerce")

FAM_EDU_map  = {
   'No, my family generally does not provide guidance on this':2,
       'Not at all, my family has never discussed this topic':1,
       'I have received some guidance, but not frequently':3,
       'Yes, I frequently receive family education or guidance':5,
       'Yes, I occasionally receive guidance from family members':4
}
merged_df.loc[:, 'FAM_EDU'] = merged_df['FAM_EDU'].replace(FAM_EDU_map)
merged_df["FAM_EDU"] = pd.to_numeric(merged_df["FAM_EDU"], errors="coerce")

PAR_MARR_map  = {
   'Neutral, some aspects are worth learning from, while others have shortcomings':3,
       'Somewhat appreciative, I think their marriage is generally good':4,
       'Very appreciative, their marriage is my ideal example':5,
       'Not appreciative at all, I do not want my marriage to be like theirs':1,
       'Not very appreciative, I think their marriage has many problems':2
}
merged_df.loc[:, 'PAR_MARR'] = merged_df['PAR_MARR'].replace(PAR_MARR_map)
merged_df["PAR_MARR"] = pd.to_numeric(merged_df["PAR_MARR"], errors="coerce")

CHILDHOOD_map  = {
   'Average, with both pleasant and difficult experiences':3,
       'Very unhappy, childhood was full of difficulties':1,
       'Very happy, filled with wonderful memories':5,
       'Quite happy, overall very satisfied':4,
       'Not very happy, with many unpleasant memories':2
}
merged_df.loc[:, 'CHILDHOOD'] = merged_df['CHILDHOOD'].replace(CHILDHOOD_map)
merged_df["CHILDHOOD"] = pd.to_numeric(merged_df["CHILDHOOD"], errors="coerce")

TRAD_IMPORTANCE_map  = {
   'Somewhat agree':4, 'Neither agree nor disagree':3,
       'Strongly agree':5, 'Strongly disagree':1, 'Somewhat disagree':2
}
merged_df.loc[:, 'TRAD_IMPORTANCE'] = merged_df['TRAD_IMPORTANCE'].replace(TRAD_IMPORTANCE_map)
merged_df["TRAD_IMPORTANCE"] = pd.to_numeric(merged_df["TRAD_IMPORTANCE"], errors="coerce")

IDENTITY_map  = {
   'Somewhat agree':4, 'Neither agree nor disagree':3,
       'Strongly agree':5, 'Strongly disagree':1, 'Somewhat disagree':2
}
merged_df.loc[:, 'IDENTITY'] = merged_df['IDENTITY'].replace(IDENTITY_map)
merged_df["IDENTITY"] = pd.to_numeric(merged_df["IDENTITY"], errors="coerce")

CULT_CONFIDENCE_map  = {
   'Somewhat agree':4, 'Neither agree nor disagree':3,
       'Strongly agree':5, 'Strongly disagree':1, 'Somewhat disagree':2
}
merged_df.loc[:, 'CULT_CONFIDENCE'] = merged_df['CULT_CONFIDENCE'].replace(CULT_CONFIDENCE_map)
merged_df["CULT_CONFIDENCE"] = pd.to_numeric(merged_df["CULT_CONFIDENCE"], errors="coerce")

FAMILY_INFLUENCE_map  = {
   'Somewhat agree':4, 'Neither agree nor disagree':3,
       'Strongly agree':5, 'Strongly disagree':1, 'Somewhat disagree':2
}
merged_df.loc[:, 'FAMILY_INFLUENCE'] = merged_df['FAMILY_INFLUENCE'].replace(FAMILY_INFLUENCE_map)
merged_df["FAMILY_INFLUENCE"] = pd.to_numeric(merged_df["FAMILY_INFLUENCE"], errors="coerce")

FAM_ADVICE_map  = {
   'Somewhat agree':4, 'Neither agree nor disagree':3,
       'Strongly agree':5, 'Strongly disagree':1, 'Somewhat disagree':2
}
merged_df.loc[:, 'FAM_ADVICE'] = merged_df['FAM_ADVICE'].replace(FAM_ADVICE_map)
merged_df["FAM_ADVICE"] = pd.to_numeric(merged_df["FAM_ADVICE"], errors="coerce")

ON_PATENER_ADVICE_map  = {
   'Somewhat agree':4, 'Neither agree nor disagree':3,
       'Strongly agree':5, 'Strongly disagree':1, 'Somewhat disagree':2
}
merged_df.loc[:, 'ON_PATENER_ADVICE'] = merged_df['ON_PATENER_ADVICE'].replace(ON_PATENER_ADVICE_map)
merged_df["ON_PATENER_ADVICE"] = pd.to_numeric(merged_df["ON_PATENER_ADVICE"], errors="coerce")

TRAD_MAR_map  = {
   'Somewhat agree':4, 'Neither agree nor disagree':3,
       'Strongly agree':5, 'Strongly disagree':1, 'Somewhat disagree':2
}
merged_df.loc[:, 'TRAD_MAR'] = merged_df['TRAD_MAR'].replace(TRAD_MAR_map)
merged_df["TRAD_MAR"] = pd.to_numeric(merged_df["TRAD_MAR"], errors="coerce")

PRIORITY_map  = {
   'Somewhat agree':4, 'Neither agree nor disagree':3,
       'Strongly agree':5, 'Strongly disagree':1, 'Somewhat disagree':2
}
merged_df.loc[:, 'PRIORITY'] = merged_df['PRIORITY'].replace(PRIORITY_map)
merged_df["PRIORITY"] = pd.to_numeric(merged_df["PRIORITY"], errors="coerce")

PAR_EVEN_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'PAR_EVEN'] = merged_df['PAR_EVEN'].replace(PAR_EVEN_map)
merged_df["PAR_EVEN"] = pd.to_numeric(merged_df["PAR_EVEN"], errors="coerce")

ECO_MALE_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'ECO_MALE'] = merged_df['ECO_MALE'].replace(ECO_MALE_map)
merged_df["ECO_MALE"] = pd.to_numeric(merged_df["ECO_MALE"], errors="coerce")

DECISION_MALE_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'DECISION_MALE'] = merged_df['DECISION_MALE'].replace(DECISION_MALE_map)
merged_df["DECISION_MALE"] = pd.to_numeric(merged_df["DECISION_MALE"], errors="coerce")

PAR_BOTH_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'PAR_BOTH'] = merged_df['PAR_BOTH'].replace(PAR_BOTH_map)
merged_df["PAR_BOTH"] = pd.to_numeric(merged_df["PAR_BOTH"], errors="coerce")

LABOUR_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'LABOUR'] = merged_df['LABOUR'].replace(LABOUR_map)
merged_df["LABOUR"] = pd.to_numeric(merged_df["LABOUR"], errors="coerce")

MALE_LABOUR_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'MALE_LABOUR'] = merged_df['MALE_LABOUR'].replace(MALE_LABOUR_map)
merged_df["MALE_LABOUR"] = pd.to_numeric(merged_df["MALE_LABOUR"], errors="coerce")

WORKING_GDRO_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'WORKING_GDRO'] = merged_df['WORKING_GDRO'].replace(WORKING_GDRO_map)
merged_df["WORKING_GDRO"] = pd.to_numeric(merged_df["WORKING_GDRO"], errors="coerce")


ABS_EQUAL_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'ABS_EQUAL'] = merged_df['ABS_EQUAL'].replace(ABS_EQUAL_map)
merged_df["ABS_EQUAL"] = pd.to_numeric(merged_df["ABS_EQUAL"], errors="coerce")

FE_ALL_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'FE_ALL'] = merged_df['FE_ALL'].replace(FE_ALL_map)
merged_df["FE_ALL"] = pd.to_numeric(merged_df["FE_ALL"], errors="coerce")

PREJ_map  = {
   'Neither agree nor disagree':3, 'Strongly agree':5,
       'Somewhat agree':4, 'Somewhat disagree':2, 'Strongly disagree':1
}
merged_df.loc[:, 'PREJ'] = merged_df['PREJ'].replace(PREJ_map)
merged_df["PREJ"] = pd.to_numeric(merged_df["PREJ"], errors="coerce")

In [26]:
merged_df.dtypes

ResponseId               object
Duration(inseconds)     float64
Progress                  int64
GEN                      object
AGE                     float64
FACULTY                  object
RACE                     object
BIRTH_PLACE              object
HAVE_BELIEF              object
RELIGION                 object
MARR_STATUS              object
BIOKIDS                  object
HAVE_SIBLINGS            object
BIRTH_ORDER            category
WHO_SIBLINGS             object
BIO_SIBLINGS             object
CLOSE_FAMILY             object
CLOSE_FAMILY_NS          object
FATHER_EDU             category
MOTHER_EDU             category
INCOME_SOURCE            object
INCOME                  float64
Duration(inseconds)     float64
Progress                float64
R_STATUS               category
TIMES                  category
IDEA_AGE               category
FIRST_AGE              category
PURPOSE                  object
REASON                   object
CRITERIA                 object
VIEW    

In [27]:
def categorize_variables(df):
    dtypes = df.dtypes
    
    discrete_variables = []
    continuous_variables = []
    
    for column, dtype in dtypes.items():
        if dtype == 'object' or dtype == 'category':
            discrete_variables.append(column)
        elif pd.api.types.is_numeric_dtype(dtype):
            continuous_variables.append(column)
    
    return discrete_variables, continuous_variables

discrete_variables, continuous_variables = categorize_variables(merged_df)

# Print the results
print("Discrete Variables:")
print(discrete_variables)
print("\nContinuous Variables:")
print(continuous_variables)

Discrete Variables:
['ResponseId', 'GEN', 'FACULTY', 'RACE', 'BIRTH_PLACE', 'HAVE_BELIEF', 'RELIGION', 'MARR_STATUS', 'BIOKIDS', 'HAVE_SIBLINGS', 'BIRTH_ORDER', 'WHO_SIBLINGS', 'BIO_SIBLINGS', 'CLOSE_FAMILY', 'CLOSE_FAMILY_NS', 'FATHER_EDU', 'MOTHER_EDU', 'INCOME_SOURCE', 'R_STATUS', 'TIMES', 'IDEA_AGE', 'FIRST_AGE', 'PURPOSE', 'REASON', 'CRITERIA', 'VIEW', 'EXPECTATION_SINGLE', 'EXPECTATION', 'R_STATUS', 'MAR_AGE', 'EXCPET_MAR', 'R_STATUS', 'CHILD_GEN', 'FACTOR_FERT_PLAN', 'FERT_PURPOSE', 'FERT_CAREER', 'WHO_PARENT', 'CHILD_NUM_INFLUENCE', 'PARENT_RESPONS', 'FACTOR_FERT']

Continuous Variables:
['Duration(inseconds)', 'Progress', 'AGE', 'INCOME', 'Duration(inseconds)', 'Progress', 'AFFECT', 'EVALUATE_MAN', 'EVALUATE_BESELF', 'EVALUATE_NEGOTIATE', 'EVALUATE_EMO', 'EVALUATE_HOPE', 'Duration(inseconds)', 'Progress', 'WILLING_MAR', 'MAR_CHOICE', 'MAR_IMPORTANCE', 'LOVE_IN_MAR', 'FIN_IN_MAR', 'TRAD_IN_MAR', 'COR_IN_MRA', 'FULFIL_IN_MRA', 'ECO', 'COM', 'PER', 'EMO', 'INT', 'LIF', 'BEFORE_SE

In [28]:
control_variables = list(set(['GEN', 'AGE', 'FACULTY', 'RACE',
       'BIRTH_PLACE', 'HAVE_BELIEF',
       'RELIGION', 'MARR_STATUS', 
       'BIOKIDS', 'HAVE_SIBLINGS', 'BIRTH_ORDER', 
       'WHO_SIBLINGS', 'BIO_SIBLINGS',
       'CLOSE_FAMILY', 
       'CLOSE_FAMILY_NS', 'FATHER_EDU', 'MOTHER_EDU',
       'INCOME_SOURCE', 'INCOME', 'CULT_MARR_ATT', 'CULT_FERT_ATT', 'FAMILY_IMPACT', 'WEDDING',
       'FAMILY_BOOK', 'FAMILY_RESOURCE', 'PARICIPATION', 'STUDY_HABIT',
       'MEDIA', 'SOCIAL', 'TOPIC', 'DIVERSITY', 'FAM_EDU', 'PAR_MARR', 'CHILDHOOD',
       'TRAD_IMPORTANCE', 'IDENTITY', 'CULT_CONFIDENCE', 'FAMILY_INFLUENCE',
       'FAM_ADVICE', 'ON_PATENER_ADVICE', 'TRAD_MAR', 'PRIORITY', 'PAR_EVEN', 'ECO_MALE',
       'DECISION_MALE', 'PAR_BOTH', 'LABOUR', 'MALE_LABOUR', 'WORKING_GDRO',
       'ABS_EQUAL', 'FE_ALL', 'PREJ']))

variables = list(set(['R_STATUS', 'TIMES', 'IDEA_AGE', 'FIRST_AGE',
       'PURPOSE', 'REASON', 
       'CRITERIA', 'VIEW', 
       'EXPECTATION_SINGLE', 'EXPECTATION', 'AFFECT',
       'EVALUATE_MAN', 'EVALUATE_BESELF',
       'EVALUATE_NEGOTIATE', 'EVALUATE_EMO', 'EVALUATE_HOPE', 'R_STATUS', 
       'PLAN_FERT', 'PARTNER_PLAN_FERT', 'SCALE_PLAN_FERT', 'CHILD_NUM',
       'DISCUSS_CHILD', 'BEST_FERT_AGE', 'CHILD_GEN',
       'FACTOR_FERT_PLAN', 
       'FERT_PURPOSE', 'FERT_VALUE', 'FERT_CAREER',
       'WHO_PARENT', 'CHILD_NUM_INFLUENCE',
       'PARENT_RESPONS', 'FACTOR_FERT',
       'R_STATUS', 
       'WILLING_MAR', 'MAR_AGE', 'MAR_CHOICE',
       'MAR_IMPORTANCE', 'LOVE_IN_MAR', 'FIN_IN_MAR', 'TRAD_IN_MAR',
       'COR_IN_MRA', 'FULFIL_IN_MRA', 'ECO', 'COM', 'PER', 'EMO', 'INT', 'LIF',
       'BEFORE_SEX', 'PARTNER_BEFORE_SEX',
       'R_BEFORE_SEX', 
       'ARRANGED_MAR', 'EXCPET_MAR']))

In [29]:
def subcategorize_variables(discrete_variables, continuous_variables, control_variables, variables):
    # Convert lists to sets for efficient intersection operations
    discrete_set = set(discrete_variables)
    continuous_set = set(continuous_variables)
    control_set = set(control_variables)
    variables_set = set(variables)
    
    # Find intersections for discrete variables
    control_discrete = list(discrete_set.intersection(control_set))
    variables_discrete = list(discrete_set.intersection(variables_set))
    
    # Find intersections for continuous variables
    control_continuous = list(continuous_set.intersection(control_set))
    variables_continuous = list(continuous_set.intersection(variables_set))
    
    # Sort lists for consistency
    control_discrete.sort()
    variables_discrete.sort()
    control_continuous.sort()
    variables_continuous.sort()
    
    return control_discrete, variables_discrete, control_continuous, variables_continuous

control_discrete, variables_discrete, control_continuous, variables_continuous = subcategorize_variables(
    discrete_variables,
    continuous_variables,
    control_variables,
    variables
)

# Print results
print("Control Discrete Variables:")
print(control_discrete)
print("\nMain Discrete Variables:")
print(variables_discrete)
print("\nControl Continuous Variables:")
print(control_continuous)
print("\nMain Continuous Variables:")
print(variables_continuous)

print("\nCounts Summary:")
print(f"Control Discrete: {len(control_discrete)}")
print(f"Main Discrete: {len(variables_discrete)}")
print(f"Control Continuous: {len(control_continuous)}")
print(f"Main Continuous: {len(variables_continuous)}")

Control Discrete Variables:
['BIOKIDS', 'BIO_SIBLINGS', 'BIRTH_ORDER', 'BIRTH_PLACE', 'CLOSE_FAMILY', 'CLOSE_FAMILY_NS', 'FACULTY', 'FATHER_EDU', 'GEN', 'HAVE_BELIEF', 'HAVE_SIBLINGS', 'INCOME_SOURCE', 'MARR_STATUS', 'MOTHER_EDU', 'RACE', 'RELIGION', 'WHO_SIBLINGS']

Main Discrete Variables:
['CHILD_GEN', 'CHILD_NUM_INFLUENCE', 'CRITERIA', 'EXCPET_MAR', 'EXPECTATION', 'EXPECTATION_SINGLE', 'FACTOR_FERT', 'FACTOR_FERT_PLAN', 'FERT_CAREER', 'FERT_PURPOSE', 'FIRST_AGE', 'IDEA_AGE', 'MAR_AGE', 'PARENT_RESPONS', 'PURPOSE', 'REASON', 'R_STATUS', 'TIMES', 'VIEW', 'WHO_PARENT']

Control Continuous Variables:
['ABS_EQUAL', 'AGE', 'CHILDHOOD', 'CULT_CONFIDENCE', 'CULT_FERT_ATT', 'CULT_MARR_ATT', 'DECISION_MALE', 'DIVERSITY', 'ECO_MALE', 'FAMILY_BOOK', 'FAMILY_IMPACT', 'FAMILY_INFLUENCE', 'FAMILY_RESOURCE', 'FAM_ADVICE', 'FAM_EDU', 'FE_ALL', 'IDENTITY', 'INCOME', 'LABOUR', 'MALE_LABOUR', 'MEDIA', 'ON_PATENER_ADVICE', 'PARICIPATION', 'PAR_BOTH', 'PAR_EVEN', 'PAR_MARR', 'PREJ', 'PRIORITY', 'SOCIAL'

In [30]:
multis = ['INCOME_SOURCE',
 'PURPOSE',
 'REASON',
 'CRITERIA',
 'FACTOR_FERT_PLAN',
 'FERT_PURPOSE',
 'PARENT_RESPONS',
 'FACTOR_FERT',
 'WHO_SIBLINGS',
 'CLOSE_FAMILY',
 'CLOSE_FAMILY_NS', 'EXCEPT_MAR']
control_discrete = [x for x in control_discrete if x not in multis]
variables_discrete = [x for x in variables_discrete if x not in multis]

In [31]:
#control_discrete & variables_discrete
dropdown_cd = widgets.Dropdown(
    options=control_discrete,
    value="GEN",
    description="Control"
)
dropdown_cd

Dropdown(description='Control', index=6, options=('BIOKIDS', 'BIO_SIBLINGS', 'BIRTH_ORDER', 'BIRTH_PLACE', 'FA…

In [32]:
dropdown_vd = widgets.Dropdown(
    options=variables_discrete,
    value="CHILD_GEN",
    description="Vairable"
)

dropdown_vd

Dropdown(description='Vairable', options=('CHILD_GEN', 'CHILD_NUM_INFLUENCE', 'EXCPET_MAR', 'EXPECTATION', 'EX…

In [33]:
def calculate_association_measures(cont_table, chi2, n, p_value):
    r, c = cont_table.shape
    
    # Calculate Phi coefficient (only for 2x2 tables)
    try:
        phi = np.sqrt(chi2 / n) if r == 2 and c == 2 else None
    except:
        phi = None
    
    try:
        denominator = n * (min(r, c) - 1)
        if denominator == 0:
            v = None
        else:
            v = np.sqrt(chi2 / denominator)
    except:
        v = None
    
    if phi is not None:
        if abs(phi) < 0.1:
            phi_interpretation = "negligible"
        elif abs(phi) < 0.2:
            phi_interpretation = "weak"
        elif abs(phi) < 0.4:
            phi_interpretation = "moderate"
        else:
            phi_interpretation = "strong"
    
    if v is not None:
        if v < 0.1:
            v_interpretation = "negligible"
        elif v < 0.2:
            v_interpretation = "weak"
        elif v < 0.4:
            v_interpretation = "moderate"
        else:
            v_interpretation = "strong"

    results = "\nAssociation Measures:"
    if phi is not None:
        results += f"\nPhi Coefficient: {phi:.3f} ({phi_interpretation} association)"
    
    if v is not None:
        results += f"\nCramér's V: {v:.3f} ({v_interpretation} association)"
    else:
        results += "\nCramér's V could not be calculated"
    
    if p_value < 0.001:
        results += "\nThe association is statistically significant (p < 0.001)"
    elif p_value < 0.01:
        results += "\nThe association is statistically significant (p < 0.01)"
    elif p_value < 0.05:
        results += "\nThe association is statistically significant (p < 0.05)"
    else:
        results += "\nThe association is not statistically significant (p ≥ 0.05)"
    
    return results

def create_cross_analysis(df, var1, var2):
    cont_table = pd.crosstab(df[var1], df[var2])
    prop_table = pd.crosstab(df[var1], df[var2], normalize='index') * 100
    
    chi2, p_value, dof, expected = chi2_contingency(cont_table)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    prop_table.plot(kind='bar', stacked=True, ax=ax1)
    ax1.set_title('Stacked Bar Chart of Proportions')
    ax1.set_xlabel(var1)
    ax1.set_ylabel('Percentage')
    ax1.legend(title=var2)
    
    sns.heatmap(cont_table, 
                annot=True, 
                fmt='d',
                cmap='YlOrRd',
                ax=ax2)
    ax2.set_title('Frequency Heatmap')
    
    plt.tight_layout()
    
    return cont_table, prop_table, chi2, p_value, fig

def display_results(df, var1, var2):
    cont_table, prop_table, chi2, p_value, fig = create_cross_analysis(df, var1, var2)
    
    print("\nContingency Table (Frequencies):")
    display(cont_table)
    
    print("\nContingency Table (Percentages):")
    display(prop_table.round(2))
    
    print(f"\nChi-square test results:")
    print(f"Chi-square statistic: {chi2:.2f}")
    print(f"p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("There is a significant relationship between the variables (p < 0.05)")
    else:
        print("There is no significant relationship between the variables (p ≥ 0.05)")
    
    n = cont_table.sum().sum()
    
    association_results = calculate_association_measures(cont_table, chi2, n, p_value)
    print(association_results)
    
    plt.show()

@interact
def display_d2d(var1 = dropdown_cd, var2 = dropdown_vd):
    display_results(merged_df, dropdown_cd.value, dropdown_vd.value)

interactive(children=(Dropdown(description='Control', index=6, options=('BIOKIDS', 'BIO_SIBLINGS', 'BIRTH_ORDE…

In [34]:
dropdown_vc = widgets.Dropdown(
    options=variables_continuous,
    value="AFFECT",
    description="Vairable"
)

dropdown_vc

Dropdown(description='Vairable', options=('AFFECT', 'ARRANGED_MAR', 'BEFORE_SEX', 'BEST_FERT_AGE', 'CHILD_NUM'…

In [35]:
dropdown_cd = widgets.Dropdown(
    options=control_discrete,
    value="GEN",
    description="Control"
)
dropdown_cd

Dropdown(description='Control', index=6, options=('BIOKIDS', 'BIO_SIBLINGS', 'BIRTH_ORDER', 'BIRTH_PLACE', 'FA…

In [36]:
dropdown_cc = widgets.Dropdown(
    options=control_continuous,
    value="ABS_EQUAL",
    description="Control"
)
dropdown_cc

Dropdown(description='Control', options=('ABS_EQUAL', 'AGE', 'CHILDHOOD', 'CULT_CONFIDENCE', 'CULT_FERT_ATT', …

In [37]:
dropdown_vd = widgets.Dropdown(
    options=variables_discrete,
    value='CHILD_GEN',
    description="Vairable"
)

dropdown_vd

Dropdown(description='Vairable', options=('CHILD_GEN', 'CHILD_NUM_INFLUENCE', 'EXCPET_MAR', 'EXPECTATION', 'EX…

In [38]:
def ensure_numeric(df, continuous_var):
    if not pd.api.types.is_numeric_dtype(df[continuous_var]):
        try:
            df[continuous_var] = pd.to_numeric(df[continuous_var], errors='coerce')
        except:
            raise TypeError(f"Cannot convert {continuous_var} to numeric type")
    return df

def calculate_descriptive_stats(df, discrete_var, continuous_var):
    df = ensure_numeric(df, continuous_var)
    
    try:
        stats_df = df.groupby(discrete_var)[continuous_var].agg([
            ('count', 'count'),
            ('mean', 'mean'),
            ('std', 'std'),
            ('median', 'median'),
            ('min', 'min'),
            ('max', 'max'),
            ('Q1', lambda x: x.quantile(0.25)),
            ('Q3', lambda x: x.quantile(0.75))
        ]).round(2)
        
        return stats_df
    except Exception as e:
        print(f"Error in calculate_descriptive_stats: {str(e)}")
        print(f"Data types: {df[continuous_var].dtype}")
        return pd.DataFrame()

def check_normality(df, discrete_var, continuous_var):
    """
    Perform Shapiro-Wilk test for each group
    """
    df = ensure_numeric(df, continuous_var)
    normality_results = {}
    
    for group in df[discrete_var].unique():
        group_data = df[df[discrete_var] == group][continuous_var].dropna()
        if len(group_data) < 3:
            stat, p_value = np.nan, np.nan
        else:
            try:
                stat, p_value = stats.shapiro(group_data)
            except Exception:
                stat, p_value = np.nan, np.nan
        normality_results[group] = {'statistic': stat, 'p_value': p_value}
    
    return pd.DataFrame(normality_results).T

def perform_statistical_tests(df, discrete_var, continuous_var, normal_dist=False):
    df = ensure_numeric(df, continuous_var)
    df = df.dropna(subset=[continuous_var])
    n_groups = df[discrete_var].nunique()
    
    try:
        if normal_dist:
            if n_groups == 2:
                # Independent t-test
                groups = df[discrete_var].unique()
                group1 = df[df[discrete_var] == groups[0]][continuous_var]
                group2 = df[df[discrete_var] == groups[1]][continuous_var]
                t_stat, p_value = stats.ttest_ind(group1, group2)
                
                # Calculate Eta squared
                ss_between = np.sum((df.groupby(discrete_var)[continuous_var].mean() - 
                                   df[continuous_var].mean())**2)
                ss_total = np.sum((df[continuous_var] - df[continuous_var].mean())**2)
                eta_squared = ss_between / ss_total
                
                return {
                    'test': 'Independent t-test',
                    't_statistic': t_stat,
                    'p_value': p_value,
                    'eta_squared': eta_squared
                }
            else:
                # One-way ANOVA
                result = pg.anova(data=df, dv=continuous_var, between=discrete_var)
                eta_squared = result['ng2'][0]
                
                return {
                    'test': 'One-way ANOVA',
                    'f_statistic': result['F'][0],
                    'p_value': result['p-unc'][0],
                    'eta_squared': eta_squared
                }
        else:
            if n_groups == 2:
                # Mann-Whitney U test
                groups = df[discrete_var].unique()
                group1 = df[df[discrete_var] == groups[0]][continuous_var]
                group2 = df[df[discrete_var] == groups[1]][continuous_var]
                stat, p_value = stats.mannwhitneyu(group1, group2, alternative='two-sided')
                return {
                    'test': 'Mann-Whitney U test',
                    'statistic': stat,
                    'p_value': p_value
                }
            else:
                # Kruskal-Wallis test
                stat, p_value = stats.kruskal(*[group[continuous_var].values 
                                              for name, group in df.groupby(discrete_var)])
                return {
                    'test': 'Kruskal-Wallis test',
                    'statistic': stat,
                    'p_value': p_value
                }
    except Exception as e:
        print(f"Error in statistical tests: {str(e)}")
        return {'test': 'Error', 'error_message': str(e)}

def create_violin_plot(df, discrete_var, continuous_var):
    """
    Create violin plot with individual points and box plot inside
    """
    df = ensure_numeric(df, continuous_var)
    plt.figure(figsize=(10, 6))
    sns.violinplot(x=discrete_var, y=continuous_var, data=df, inner='box')
    plt.title(f'Distribution of {continuous_var} by {discrete_var}')
    plt.xticks(rotation=45)
    return plt.gcf()

In [39]:
def interpret_descriptive_stats(desc_stats, continuous_var):
    """
    Provide interpretation of descriptive statistics
    """
    interpretations = []
    
    # Overall patterns
    highest_group = desc_stats['mean'].idxmax()
    lowest_group = desc_stats['mean'].idxmin()
    
    interpretations.append(f"\nDescriptive Statistics Interpretation:")
    interpretations.append(f"• The highest mean {continuous_var} is found in the '{highest_group}' group ({desc_stats.loc[highest_group, 'mean']:.2f})")
    interpretations.append(f"• The lowest mean {continuous_var} is found in the '{lowest_group}' group ({desc_stats.loc[lowest_group, 'mean']:.2f})")
    
    # Variability
    most_variable = desc_stats['std'].idxmax()
    interpretations.append(f"• The group showing most variability is '{most_variable}' (SD = {desc_stats.loc[most_variable, 'std']:.2f})")
    
    # Sample sizes
    total_n = desc_stats['count'].sum()
    interpretations.append(f"• Total sample size is {total_n} observations")
    
    return "\n".join(interpretations)

def interpret_normality(normality_results):
    """
    Provide interpretation of normality tests
    """
    interpretations = ["\nNormality Test Interpretation:"]
    
    for group in normality_results.index:
        p_value = normality_results.loc[group, 'p_value']
        if pd.isna(p_value):
            interpretations.append(f"• {group}: Insufficient data for normality test")
        else:
            is_normal = p_value > 0.05
            interpretations.append(
                f"• {group}: Distribution is {'normal' if is_normal else 'not normal'} "
                f"(p = {p_value:.4f})"
            )
    
    overall_normal = all(p > 0.05 for p in normality_results['p_value'].dropna())
    interpretations.append(
        f"\nOverall: The data is {'normally' if overall_normal else 'not normally'} distributed "
        f"across groups"
    )
    
    return "\n".join(interpretations)

def interpret_statistical_test(test_results, desc_stats):
    """
    Provide interpretation of statistical test results
    """
    interpretations = ["\nStatistical Test Interpretation:"]
    
    # Get test type and p-value
    test_type = test_results['test']
    p_value = test_results.get('p_value', None)
    
    # Basic interpretation of significance
    if p_value is not None:
        significant = p_value < 0.05
        interpretations.append(
            f"• The {test_type} shows {'a significant' if significant else 'no significant'} "
            f"difference between groups (p = {p_value:.4f})"
        )
    
    # Effect size interpretation for parametric tests
    if 'eta_squared' in test_results:
        eta_squared = test_results['eta_squared']
        effect_size = "small" if eta_squared < 0.06 else \
                     "medium" if eta_squared < 0.14 else "large"
        
        interpretations.append(
            f"• Effect size (η²) = {eta_squared:.4f}, indicating a {effect_size} effect"
        )
        interpretations.append(
            f"• This means that {(eta_squared * 100):.1f}% of the variance in the outcome "
            f"can be explained by group membership"
        )
    
    # Additional interpretation based on test type
    if test_type == 'Independent t-test':
        t_stat = test_results['t_statistic']
        interpretations.append(f"• t-statistic = {t_stat:.4f}")
    elif test_type == 'One-way ANOVA':
        f_stat = test_results['f_statistic']
        interpretations.append(f"• F-statistic = {f_stat:.4f}")
    
    # Practical significance
    if significant:
        group_means = desc_stats['mean']
        max_diff = group_means.max() - group_means.min()
        interpretations.append(
            f"• The maximum difference between group means is {max_diff:.2f} units"
        )
    
    return "\n".join(interpretations)

def display_results(df, discrete_var, continuous_var):
    try:
        # Convert continuous variable to numeric if needed
        df = ensure_numeric(df, continuous_var)
        
        # Descriptive statistics
        print("\n=== DESCRIPTIVE STATISTICS ===")
        desc_stats = calculate_descriptive_stats(df, discrete_var, continuous_var)
        display(desc_stats)
        print(interpret_descriptive_stats(desc_stats, continuous_var))
        
        # Normality tests
        print("\n=== NORMALITY TEST RESULTS ===")
        normality = check_normality(df, discrete_var, continuous_var)
        display(normality)
        print(interpret_normality(normality))
        
        # Determine if distribution is normal
        is_normal = all(p > 0.05 for p in normality['p_value'].dropna())
        
        # Statistical tests
        print("\n=== STATISTICAL TEST RESULTS ===")
        test_results = perform_statistical_tests(df, discrete_var, continuous_var, is_normal)
        print(f"Test performed: {test_results['test']}")
        for key, value in test_results.items():
            if key != 'test':
                if isinstance(value, float):
                    print(f"{key}: {value:.4f}")
                else:
                    print(f"{key}: {value}")
        
        print(interpret_statistical_test(test_results, desc_stats))
        
        # Create and display violin plot
        print("\n=== VISUALIZATION ===")
        fig = create_violin_plot(df, discrete_var, continuous_var)
        plt.show()
    
    except Exception as e:
        print(f"Error in display_results: {str(e)}")

In [40]:
@interact
def display_cd(continous_var = dropdown_cc, discrete_var = dropdown_vd):
    display_results(merged_df, discrete_var, continous_var)

interactive(children=(Dropdown(description='Control', options=('ABS_EQUAL', 'AGE', 'CHILDHOOD', 'CULT_CONFIDEN…

In [41]:
@interact
def display_dc(discrete_var = dropdown_cd, continous_var = dropdown_vc):
    display_results(merged_df, dropdown_cd.value, dropdown_vc.value)

interactive(children=(Dropdown(description='Control', index=6, options=('BIOKIDS', 'BIO_SIBLINGS', 'BIRTH_ORDE…

In [42]:
def clean_data_for_analysis(df, var1, var2):
    df_clean = df[[var1, var2]].copy()

    df_clean = df_clean.replace([np.inf, -np.inf], np.nan)
    df_clean = df_clean.dropna()
    
    return df_clean

def perform_bivariate_analysis(df, var1, var2):
    df_clean = clean_data_for_analysis(df, var1, var2)
    
    if len(df_clean) == 0:
        raise ValueError("No valid data points remain after cleaning")

    try:
        pearson_corr, p_value_pearson = stats.pearsonr(df_clean[var1], df_clean[var2])
        spearman_corr, p_value_spearman = stats.spearmanr(df_clean[var1], df_clean[var2])
        covariance = np.cov(df_clean[var1], df_clean[var2])[0][1]
    except Exception as e:
        print(f"Error in calculating correlations: {str(e)}")
        return None, None

    fig = plt.figure(figsize=(20, 10))
    gs = fig.add_gridspec(2, 3)
    
    try:
        # 1. Scatter plot with regression line
        ax1 = fig.add_subplot(gs[0, 0])
        sns.regplot(data=df_clean, x=var1, y=var2, ax=ax1, scatter_kws={'alpha':0.5})
        ax1.set_title('Scatter Plot with Regression Line')
        
        # 2. 2D Kernel Density Plot
        ax2 = fig.add_subplot(gs[0, 1])
        sns.kdeplot(data=df_clean, x=var1, y=var2, ax=ax2, cmap='viridis', fill=True)
        ax2.set_title('2D Kernel Density Plot')
        
        # 3. Joint Plot
        ax3 = fig.add_subplot(gs[0, 2])
        df_sample = df_clean.sample(n=min(1000, len(df_clean)))
        sns.histplot(data=df_sample, x=var1, y=var2, bins=30, ax=ax3)
        ax3.set_title('Joint Distribution Plot')
        
        # 4. Clustering Analysis
        ax4 = fig.add_subplot(gs[1, 0])
        X = df_clean[[var1, var2]].values
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        kmeans = KMeans(n_clusters=3, random_state=42)
        clusters = kmeans.fit_predict(X_scaled)
        scatter = ax4.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis')
        ax4.set_title('K-means Clustering (k=3)')
        ax4.set_xlabel(var1)
        ax4.set_ylabel(var2)
        plt.colorbar(scatter, ax=ax4)
        
        # 5. PCA Analysis
        ax5 = fig.add_subplot(gs[1, 1])
        pca = PCA(n_components=2)
        pca_result = pca.fit_transform(X_scaled)
        ax5.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.5)
        ax5.set_title('PCA Projection')
        ax5.set_xlabel('First Principal Component')
        ax5.set_ylabel('Second Principal Component')
        
        # 6. Statistical Summary
        ax6 = fig.add_subplot(gs[1, 2])
        ax6.axis('off')
        stats_text = (
            f'Statistical Summary:\n\n'
            f'Number of valid observations: {len(df_clean)}\n'
            f'Removed NA/inf values: {len(df) - len(df_clean)}\n\n'
            f'Pearson Correlation: {pearson_corr:.3f}\n'
            f'Pearson P-value: {p_value_pearson:.3e}\n\n'
            f'Spearman Correlation: {spearman_corr:.3f}\n'
            f'Spearman P-value: {p_value_spearman:.3e}\n\n'
            f'Covariance: {covariance:.3f}\n\n'
            f'PCA Explained Variance Ratio:\n'
            f'PC1: {pca.explained_variance_ratio_[0]:.3f}\n'
            f'PC2: {pca.explained_variance_ratio_[1]:.3f}'
        )
        ax6.text(0.1, 0.9, stats_text, transform=ax6.transAxes, 
                verticalalignment='top', fontfamily='monospace')
        
        plt.tight_layout()
        
    except Exception as e:
        print(f"Error in creating visualizations: {str(e)}")
        return None, None
    
    return fig, {
        'pearson': pearson_corr,
        'spearman': spearman_corr,
        'covariance': covariance,
        'pca_variance_ratio': pca.explained_variance_ratio_,
        'n_valid': len(df_clean),
        'n_removed': len(df) - len(df_clean)
    }

In [43]:
def interpret_correlation(value, p_value):
    if abs(value) < 0.1:
        strength = "negligible"
    elif abs(value) < 0.3:
        strength = "weak"
    elif abs(value) < 0.5:
        strength = "moderate"
    elif abs(value) < 0.7:
        strength = "strong"
    else:
        strength = "very strong"
    
    direction = "positive" if value > 0 else "negative"
    
    significance = "statistically significant" if p_value < 0.05 else "not statistically significant"
    
    return f"The correlation is {strength} and {direction} ({value:.3f}) and is {significance} (p={p_value:.3e})"

def interpret_pca(variance_ratios):
    total_explained = sum(variance_ratios) * 100
    return f"The first two principal components explain {total_explained:.1f}% of the total variance " \
           f"(PC1: {variance_ratios[0]*100:.1f}%, PC2: {variance_ratios[1]*100:.1f}%)"

In [44]:
def display_results(df, var1, var2):
    plt.close('all')
    
    try:
        fig, stats_dict = perform_bivariate_analysis(df, var1, var2)
        
        if fig is None or stats_dict is None:
            print("Analysis could not be completed due to data issues.")
            return
            
        removal_percentage = stats_dict['n_removed']/len(df)*100
        if removal_percentage > 50:
            print("\n⚠️ WARNING: Data Quality Issue")
            print("=" * 80)
            print(f"A very high proportion ({removal_percentage:.1f}%) of observations were removed due to missing or invalid values.")
            print("This may significantly impact the reliability of the analysis.")
            print("Recommendations:")
            print("1. Investigate why so many values are missing/invalid")
            print("2. Consider data imputation techniques if appropriate")
            print("3. Review data collection process")
            print("4. Interpret results with extreme caution")
            print("\n(Note: Following analysis results may not be reliable)")
            print("=" * 80 + "\n")

        print(f"\nDetailed Analysis Results: {var1} vs {var2}")
        print("=" * 80)
        
        print("\n1. Data Quality")
        print("-" * 80)
        print(f"• Total observations analyzed: {stats_dict['n_valid']}")
        print(f"• Removed observations: {stats_dict['n_removed']} ({removal_percentage:.1f}% of total)")
        
        print("\n2. Correlation Analysis")
        print("-" * 80)
        print("Pearson Correlation:")
        print(f"• Correlation coefficient: {stats_dict['pearson']:.3f}")
        
        print("\nSpearman Correlation:")
        print(f"• Correlation coefficient: {stats_dict['spearman']:.3f}")
        
        for corr_type, corr_value in [("Pearson", stats_dict['pearson']), 
                                     ("Spearman", stats_dict['spearman'])]:
            strength = ""
            if abs(corr_value) < 0.1:
                strength = "negligible"
            elif abs(corr_value) < 0.3:
                strength = "weak"
            elif abs(corr_value) < 0.5:
                strength = "moderate"
            elif abs(corr_value) < 0.7:
                strength = "strong"
            else:
                strength = "very strong"
                
            direction = "positive" if corr_value > 0 else "negative"
            print(f"• {corr_type} correlation indicates a {strength} {direction} relationship")
            
        print("\n3. Covariance")
        print("-" * 80)
        print(f"• Covariance: {stats_dict['covariance']:.3f}")
        print("• Note: The scale of covariance depends on the units of measurement")
        
        print("\n4. Principal Component Analysis")
        print("-" * 80)
        total_explained = sum(stats_dict['pca_variance_ratio']) * 100
        print(f"• Total variance explained by first two components: {total_explained:.1f}%")
        print(f"• First component explains: {stats_dict['pca_variance_ratio'][0]*100:.1f}%")
        print(f"• Second component explains: {stats_dict['pca_variance_ratio'][1]*100:.1f}%")
        
        print("\n5. Visual Analysis Guide")
        print("-" * 80)
        print("The following plots are provided for visual analysis:")
        print("• Scatter Plot: Examine the overall relationship pattern")
        print("• Density Plot: Shows the concentration of observations")
        print("• Joint Distribution: Shows marginal distributions")
        print("• Clustering Plot: Identifies potential natural groupings")
        print("• PCA Plot: Shows data structure in reduced dimensions")
        
        plt.show()

        print("\n6. Linear Regression Analysis")
        print("=" * 80)
        
        # Perform regression analysis
        regression_results = perform_linear_regression(df, var1, var2)
        
        # Display regression interpretation
        print(interpret_regression_results(regression_results))
        
        # Show regression diagnostic plots
        plt.show()
        
        # Display detailed model summary
        print("\n7. Detailed Model Summary")
        print("=" * 80)
        print(regression_results['model_summary'])
        
        if stats_dict['n_valid'] < 30:
            print("\n⚠️ Note: Small Sample Size")
            print("The analysis is based on a small sample size (<30 observations).")
            print("Results should be interpreted with caution.")
            
    except Exception as e:
        print(f"Error in displaying results: {str(e)}")
        print("Please check if the selected variables contain valid numerical data.")

In [45]:
def perform_linear_regression(df, var1, var2):
    df_clean = clean_data_for_analysis(df, var1, var2)
    
    if len(df_clean) == 0:
        raise ValueError("No valid data points remain after cleaning")

    X = df_clean[var1].values.reshape(-1, 1)
    y = df_clean[var2].values

    X_with_const = sm.add_constant(X)
    model = sm.OLS(y, X_with_const).fit()

    predictions = model.predict(X_with_const)
    residuals = model.resid
    standardized_residuals = model.resid_pearson

    r2 = r2_score(y, predictions)
    rmse = np.sqrt(mean_squared_error(y, predictions))

    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Actual vs Predicted Plot
    axes[0, 0].scatter(predictions, y, alpha=0.5)
    axes[0, 0].plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
    axes[0, 0].set_xlabel('Predicted Values')
    axes[0, 0].set_ylabel('Actual Values')
    axes[0, 0].set_title('Actual vs Predicted Values')
    
    # 2. Residuals vs Predicted Plot
    axes[0, 1].scatter(predictions, residuals, alpha=0.5)
    axes[0, 1].axhline(y=0, color='r', linestyle='--')
    axes[0, 1].set_xlabel('Predicted Values')
    axes[0, 1].set_ylabel('Residuals')
    axes[0, 1].set_title('Residuals vs Predicted Values')
    
    # 3. Q-Q Plot
    stats.probplot(standardized_residuals, dist="norm", plot=axes[1, 0])
    axes[1, 0].set_title('Normal Q-Q Plot')
    
    # 4. Scale-Location Plot
    axes[1, 1].scatter(predictions, np.sqrt(np.abs(standardized_residuals)), alpha=0.5)
    axes[1, 1].set_xlabel('Predicted Values')
    axes[1, 1].set_ylabel('√|Standardized Residuals|')
    axes[1, 1].set_title('Scale-Location Plot')
    
    plt.tight_layout()

    results = {
        'model_summary': model.summary(),
        'coefficients': model.params,
        'r_squared': r2,
        'adj_r_squared': model.rsquared_adj,
        'rmse': rmse,
        'f_stat': model.fvalue,
        'f_pvalue': model.f_pvalue,
        'n_observations': len(df_clean),
        'diagnostic_plots': fig
    }
    
    return results

def interpret_regression_results(results):
    interpretation = []
    
    # Model Fit
    interpretation.append("\n1. Model Fit")
    interpretation.append("-" * 80)
    interpretation.append(f"• R-squared: {results['r_squared']:.3f} "
                        f"(explains {results['r_squared']*100:.1f}% of variance)")
    interpretation.append(f"• Adjusted R-squared: {results['adj_r_squared']:.3f}")
    interpretation.append(f"• RMSE: {results['rmse']:.3f}")
    
    # Model Significance
    interpretation.append("\n2. Model Significance")
    interpretation.append("-" * 80)
    interpretation.append(f"• F-statistic: {results['f_stat']:.3f}")
    interpretation.append(f"• F-test p-value: {results['f_pvalue']:.3e}")
    
    if results['f_pvalue'] < 0.05:
        interpretation.append("• The model is statistically significant (p < 0.05)")
    else:
        interpretation.append("• The model is not statistically significant (p ≥ 0.05)")
    
    # Coefficients
    interpretation.append("\n3. Coefficients")
    interpretation.append("-" * 80)
    interpretation.append(f"• Intercept: {results['coefficients'][0]:.3f}")
    interpretation.append(f"• Slope: {results['coefficients'][1]:.3f}")
    
    # Model Diagnostics
    interpretation.append("\n4. Model Diagnostics Guide")
    interpretation.append("-" * 80)
    interpretation.append("Please examine the diagnostic plots:")
    interpretation.append("• Actual vs Predicted: Check for prediction accuracy")
    interpretation.append("• Residuals vs Predicted: Check for homoscedasticity")
    interpretation.append("• Q-Q Plot: Check for normality of residuals")
    interpretation.append("• Scale-Location: Check for constant variance")
    
    return "\n".join(interpretation)

In [46]:
@interact
def display_dc(var1 = dropdown_cc, var2 = dropdown_vc):
    display_results(merged_df, dropdown_cc.value, dropdown_vc.value)

interactive(children=(Dropdown(description='Control', options=('ABS_EQUAL', 'AGE', 'CHILDHOOD', 'CULT_CONFIDEN…